> # 0. 구성
1. 인텔을 이기다
       [내용]
        - Intel 머신러닝 프로그램 실습 도중 보다 높은 고객이탈율 예측정확도를 산출할 수 있는 지표를 발견하기까지의 과정과 그 너머
        [흐름]
        - 문제 해결
            - 문제 제시: 인텔 제공
            - 우리 방법: 내 해결안
            - 인텔 방법: 인텔 해결안
            - 차이 및 배움: 두 해결안의 차이 및 이를 통한 배움
            - 위 네 항목 사이 '잠깐 실험실' '참고학습' '중간분석' 등 관련 항목 추가
        - 결과 분석
            - 최종분석: 94.22%(인텔)와 94.34%(우리-최초모델) 정확도 차이의 원인 분석
            - 추가분석: KNN 외 모델/다른 지표로 추가 실험 및 분석 결과 100% 정확도 산출
        - 모델 전달
            - 오렌지텔레콤의 고객관리 담당자를 위한 '원클릭 고객 이탈 예측 서비스'
2. 배민라이더스 앱 리뉴얼 효과 분석 제안서
       [내용]
        - 배민라이더스 앱의 리뉴얼을 앞두고 있다는 가정 하에 그 효과에 대한 사전 검증을 위한 분석 계획 수립
        [흐름]
        - 정의 및 구체화
            - 상황 설정
            - 용어 정의
            - 목표 재정의
            - 문제 정의
            - 문제 구체화
            - 필요 자원/자료 정의
        - 조사 및 분석
            - 조사/분석 실행 전 사내 조사
            - 조사/분석 실행 시 유의사항
            - 단계별 조사/분석 실행
            - 각 후보안에 대한 판단 근거
        - 해결 및 평가
            - 최종 해결안 선정
            - 최종 해결안 실행
            - 결과 확인 및 평가

> # 1. 인텔을 이기다

## [인텔 AI 개발자 프로그램] 머신러닝 2주차 - 지도학습과 KNN

이번 주 실습에서는 통신사의 고객이탈율 데이터를 활용할 거예요. 이름은 `Orange_Telecom_Churn_Data.csv`입니다. 함께 데이터를 불러오고, 필요한 정제를 하고, 최종적으로 KNN 모델을 활용하여 각 고객별 특성에 따른 이탈 여부를 예측하고자 합니다.

### 문제 재정의 (구체화)
    1. 목표: 오렌지텔레콤을 위한 고객 이탈 여부 예측
    2. 데이터: 고객 기본 정보 / 서비스 이용 현황 / 과거 이탈 여부
    3. 제안: 예측 시 KNN을 활용

In [ ]:
# 필요환경 구성

from __future__ import print_function
import os

data_path = ['../input/intelml101class1']

## 문제 1

* 먼저 데이터를 불러오세요. 이후 지표(column)들과 데이터를 살펴보세요.
* 지표 중 주 이름(state), 지역번호(area code), 전화번호(phone number)가 포함되어 있네요.
머신러닝 모델을 설계하는 데 이것들이 좋은 지표가 될 수 있을까요? 여러분의 의견과 이유를 알려주세요.
* 저흰(Intel) 이걸 사용 안 할 겁니다. 그러니 지표 목록에서 삭제해도 되겠죠.

### 우리 방법

- 데이터 불러오기 - 사례 고객 5,000 명 중 마지막 고객 예시

In [ ]:
import numpy as np
import pandas as pd

filepath = os.sep.join(data_path + ['Orange_Telecom_Churn_Data.csv'])
#print(filepath);
data = pd.read_csv(filepath)

# 사례 고객: 5,000명
data.tail(n=1).T

- 지표별 통계수치 살펴보기

In [ ]:
# 기본적 통계수치 확인
data.describe().T

#### 기초분석 - 지표 파악
1. 타겟: 고객 이탈(True/False), 마지막 열에 있음
2. 지표 구성: 총 21 / 타겟(churned) 제외 20
    - 수치: 14
    - 분류: 1 (state)
    - True or False: 3 (intl_plan, voice_mail_plan, churned) / 타겟 제외 2
    - 숫자지만 수치는 아닌 지표: 3 (account_length, area_code, phone_number)
        - 계정 접근 시 인덱스를 활용하면 되므로 account_length, phone_number 삭제
        - state에 위치 정보가 포함되어 있으므로 area_code 삭제
3. 필요치 않은 열 삭제 후 지표 수: 18 / 타겟 제외 17

In [ ]:
# 필요없는 열 삭제
data_raw = data.copy() # 원형 데이터 저장 (의도치 않게 data 변수가 변경될 경우 대비)

col_unused = ['account_length', 'area_code', 'phone_number']
data_slim = data.drop(columns=col_unused)
#print(len(data_slim.columns)) # 18 = 21 - 3
data_slim.tail(n=1).T

### Intel 방법

In [ ]:
import pandas as pd

# 데이터 불러오기
filepath = os.sep.join(data_path + ['Orange_Telecom_Churn_Data.csv'])
data = pd.read_csv(filepath)

In [ ]:
data.head(1).T

In [ ]:
# 필요없는 열 삭제
data.drop(['state', 'area_code', 'phone_number'], axis=1, inplace=True)
data.columns

### 차이와 배움
1. 인텔: data.head(1).T - 트랜스포즈를 사용하여 전체 컬럼을 보기 쉽게 만듦
    - 우리 코드에 적용 (완료)
2. drop(inplace=True)의 기능
    - True일 때, 드롭된 버전의 새 데이터가 기존 데이터를 대체함
    - False일 때, 드롭된 버전의 새 데이터를 얻을 수 있음 (기존 것 유지)
    - False가 기본 값

## 문제 2

* 열 중 일부는 분류형 값이고 일부는 실수입니다. 수업 중 다룬 방식 중 하나를 적용하여 이 지표들을 수치화하세요.
* KNN 모델은 단일화된 수치적 데이터를 요구한다는 거 기억하시죠? 마찬가지로 수업 내 방식을 적용해서 데이터를 변환하세요.

### 우리 방법

- 분류를 숫자로 바꾸기

In [ ]:
# 전체 열 데이터 종류 확인
pd.DataFrame(data_slim).dtypes

#### 중간분석 - 수치형 제외 데이터 종류 (위에서 기초분석한 내용을 갱신)
1. 분류형: 3 (state, intl_plan, voice_mail_plan)
    - intl_plan, voice_mail_plan은 bool형이 아니라 분류형으로 확인
2. bool형: 1 (churned)
3. 숫자지만 범주에 가까운 열: number_customer_service_calls (서비스콜 수, 범주화 가능) > 다른 열 변환 후 변환 필요 확인
3. 실험적으로 state부터 변환 / factorize 활용

- 변환 전

In [ ]:
# 실험적으로 state부터 변환 / factorize 활용

# 변환 전
print(data_slim.state.tail(n=3), '\n', '\n', 'state 유일값 수: ', len(data_slim.state.unique()), '\n', data_slim.state.unique())

- 변환 후

In [ ]:
# 변환 후
data_slim.state= pd.factorize(data_slim.state)[0]
print(data_slim.state.tail(n=3), '\n', '\n', 'state 유일값 수: ', len(data_slim.state.unique()), '\n', data_slim.state.unique())

- 나머지 비숫자 열 - 변환 예시

In [ ]:
# 나머지 비숫자 열 변환
for i in range(len(data_slim.columns)):
    if data_slim.dtypes[i] != int and data_slim.dtypes[i] != float: # 숫자 여부 확인
        col = data_slim.columns[i]
        data_slim[col] = pd.factorize(data_slim[col])[0]
data_slim.tail(n=3).T # 변환된 내용

- 최종 변환 결과 - 열 종류

In [ ]:
# 변환 결과 (열 종류)
data_slim.dtypes

- 숫자지만 범주에 가까운 열 (number_customer_service_calls) 확인 후 변환 - factorize 적용

In [ ]:
# 추가적으로, 숫자지만 범주에 가까운 열 (number_customer_service_calls) 확인

# 변환 전
print('변환 전 -', data_slim.number_customer_service_calls.unique())

# 변환 가능해보임 > 해보자 > 실은 변환할 필요없었음 > 이미 숫자임!
data_slim['number_customer_service_calls'] = pd.factorize(data_slim.number_customer_service_calls)[0]

# 변환 후
print('변환 후 -', data_slim.number_customer_service_calls.unique())

#### 참고학습 - 객체 (object)
1. intl_plan / voice_mail_plan: 객체(우리의 경우 분류형)
2. 객체에 대하여
> - 객체 데이터 유형이란?
    - 객체지향언어에서 비객체형 값을 감싸서 동적 객체처럼 보이게 만드는 데이터유형
  - 동적 객체란?
    - (사전에 정의되어 프로그램 실행 내내 존재하는) 정적 객체의 반대
        - 예를 들어 동적 객체를 활용하여 일단 "물의 양"이라는 클래스를 정의내리면
            - 이후에 사용자가 새 박스에 새 물을 채울 때엔 새로운 예시를 동적으로 만들어낼 수 있음 
        - 정적 객체는 미리 이런 상황을 예측하여 사전에 그 새로운 물의 양까지 정의내려야 함
    - 하지만 intl_plan / voice_mail_plan은 오직 Y/N 두 값만 갖고 있으므로 위 필요에 맞지 않는 것으로 보임
    - state 역시 50개라는 제한된 입력값을 갖고 있음
    - 아무래도 아직 '객체'에 대해 분명한 그림이 그려지지 않은 듯함 (객체지향언어에 대해서도)
    - 그때그때 나올 때마다 좀 더 찾아보며 배워야함
3. 이제 위의 두 열이 bool이 아닌 객체인 걸 알았으니 변환 기능을 새로 써봅시다 (최초 버전: bool 여부 확인 후 변환)
    - bool 혹은 객체(분류형) 여부에 상관없이 변환하도록 코드 변경 (완료 - 숫자 여부 확인으로 일반화)

[잠깐 실험실] factorize(sort=True/False)의 차이

고객 서비스콜 수 히스토그램 (원형 데이터)

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('seaborn-dark')
plt.xlabel('No. of Customer Service Calls')
plt.ylabel('Frequency')
plt.title('Frequency by No. of Service Calls (raw data)')
plt.hist(data_raw.number_customer_service_calls, color='k', alpha=0.67);

고객 서비스콜 수 히스토그램 (factorize(sorted=True) 적용)

In [ ]:
serv_call_fact_sort = pd.factorize(data_raw.number_customer_service_calls, sort=True)[0]
print('서비스콜 수 기준, 원형 데이터와 일치하는 고객 수:', sum(serv_call_fact_sort == data_raw.number_customer_service_calls))
print('서비스콜 수 총합:', sum(serv_call_fact_sort))
plt.style.use('seaborn-dark')
plt.xlabel('No. of Customer Service Calls')
plt.ylabel('Frequency')
plt.title('Frequency by No. of Service Calls (sorted factorized)')
plt.hist(serv_call_fact_sort, color='navy', alpha=0.67);

고객 서비스콜 수 히스토그램 (factorize(sorted=False) 적용)

In [ ]:
serv_call_fact_unsort = pd.factorize(data_raw.number_customer_service_calls)[0] #false가 기본값
print('서비스콜 수 기준, 원형 데이터와 일치하는 고객 수:', sum(serv_call_fact_unsort == data_raw.number_customer_service_calls))
print('서비스콜 수 총합:', sum(serv_call_fact_unsort))
plt.style.use('seaborn-dark')
plt.xlabel('No. of Customer Service Calls')
plt.ylabel('Frequency')
plt.title('Frequency by No. of Service Calls (unsorted factorized)')
plt.hist(serv_call_fact_unsort, color='crimson', alpha=0.67);

- 결과분석 (아래에서 KNN모델 학습 후 얻은 예측정확도를 참고)
    1. number_customer_service_call열에 factorize(sorted=False) 적용 시 예측정확도 94.44% (sorted=True 적용 시 94.34%)
    2. factorize(sorted) 미적용 시 정확도 0.1% 상승 (5명 더 많은 고객에 대한 정확한 예측) 이유 추론
        - 수치형 데이터에 factorize 적용 시 sorted값을 True로 설정하지 않으면 factorize 후 인덱스와 값의 관계가 틀어짐 (False가 기본)
            - 변환 전 [1 0 2 3 4 5 7 9 6 8]이던 고객서비스 콜 수 유일값이 변환을 통해 [0 1 2 3 4 5 6 7 8 9]로 바뀜
            - 컴퓨터는 factorize 전 데이터를 수치로 보지 않고 수치화해야 할 범주로 보기 때문
        - 위의 과정을 통해 해당 고객과 상관없는 다른 고객의 서비스콜 횟수가 연결됨 (총 2,860명의 서비스콜 수 변경)
        - 전체 서비스콜 수 역시 7852에서 7142로 변경
            - 서비스콜 수 1인 고객과 0인 고객이 맞바뀜
            - 서비스콜 수 6~9 사이의 고객들의 값이 맞바뀜
        - 이렇게 얻어진 0.1%의 정확도 상승은 우연에 불과하므로 직접 sorted를 True로 설정하여 인덱스와 값의 관계를 유지해야 함
            - 전체 중 약 56% 고객의 데이터가 변경되었음에도 0.1%의 정확도만 영향을 받은 이유 추론
                - 서비스콜 수가 유일한 지표가 아님 (이외 16개의 다른 지표 존재)
                - 서비스콜 수가 유의미한 지표가 아님 (고객 이탈율과 서비스 콜 수에 상관관계가 없거나 희미함)
                    - 상관관계 그래프를 통해 반증 (아래) - 이탈율과 서비스콜 수는 유의미한 양의 상관관계를 갖는 것으로 드러남
                - 서비스콜 수가 유의미한 지표지만 위의 변동된 내용이 예측에 큰 영향을 미칠 정도는 아님
                    - 서비스콜 1과 0 / 6부터 9 사이 고객 모두 전체 중 양극단에 위치
                    - 1과 0은 서로 성향이 같고 6부터 9 사이의 고객도 서비스콜 수에 있어서 그러함
                    - 즉 0과 9의 뒤바뀜이라거나 3과 7의 뒤바뀜에 비해 그 영향이 미미하다고 볼 수 있음
            - 정확도가 하락하지 않고 상승한 이유
                - 고객 이탈율과 서비스콜 수는 양의 상관관계를 가짐
                - 전체 고객 중 약 86%는 이탈하지 않은 고객 (타겟값 0)
                - 즉 서비스콜 수 역시 전체 중 약 86%의 고객이 0에 가까운 값을 지닐 것이 기대됨
                - 위 변동을 통해 기존 콜 수가 1인 고객(약 1750명)이 0인 고객(약 1000명)과 뒤바뀜
                    - 즉 750명 많은 고객이 서비스콜을 덜 이용한 것으로 데이터가 변경됨
                    - 이 변경된 추세는 서비스콜이 0에 가까울수록 고객 이탈율이 낮다는 상관관계에 부합
                    - 즉 더 많은 고객(실제로 5명)에 대한 비이탈(타겟값 0) 예측으로 이어짐
                - 단, 6~9 영역의 고객들 사이 변동 중 6과 8의 변동에서 더 많은 고객이 6에서 8로 옮겨짐
                    - 하지만 그 수치가 50 미만으로 위 750명의 영향에 미치지 못함
                    - 이로 인해 더 많은 고객에 대한 이탈 예측이 있었다 해도 위의 비이탈 예측으로 상쇄됨
    3. sort된 버전의 factorize로 코드 변경 후, 이하 관련된 분석 내 정확도 역시 94.44%에서 94.34%로 변경 필요 (완료)

지표 간 / 지표별 타겟값과의 상관관계 그래프 (중복을 피하기 위해 표 우상단의 값은 삭제하였습니다)
    - 맨 아랫줄(churned)이 타겟값과 각 지표의 관계
    - 붉을수록 높은 양의 상관관계
    - 덜 붉을수록 낮은 양의 상관관계
    - 푸를수록 높은 음의 상관관계

In [ ]:
# 지표간 상관관계 / 지표별 고객이탈율과의 상관관계

import seaborn as sns

sns.set(style="dark")

# 상관관계 매트릭스
corr = data_raw.corr()

# 중복되는 윗부분 가리기
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(7, 5))
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# 상관관계 그래프
corr_mat = sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
                       square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
# sorted된 factorizer로 업데이트
data_slim['number_customer_service_calls'] = serv_call_fact_sort

- 실험 종료 (factorize(sort=True/False)의 차이)

#### 중간분석 - 수치 단일화(scaling) 전 고려 사항
> 제시된 세 가지 스케일링 중에서 이 문제에는 뭐가 최적일까? (각 값의 실제 계산과정 확인)
     1. standard
        - x_scaled = (x-mean) / standard deviation
     2. min-max
        - x_scaled = (x-min(x)) / (max(x)–min(x)) *기본 설정 범주 = [0,1]
     3. max-abs
        - x_scaled = x / max(abs(x))
- 아직 이 문제에 뭐가 최적인지 모르겠음 > 일단 "표준"인 standard로 실험

수치 단일화 (scaling)

- StandardScaler 적용 결과

In [ ]:
# StandardScaler
from sklearn.preprocessing import StandardScaler

"""
잠깐 메모: 파이썬식 변수 이름 짓기 (검색 후 적용)
> 계속 아래_막대_방법을 적용하자,
  작업하고 있는 환경이 엄격하게 camelCase를 따르는 게 아니라면
"""
StdSc = StandardScaler()
data_slim_std_scaled = pd.DataFrame(StdSc.fit_transform(data_slim), columns = data_slim.columns)

data_slim_std_scaled.describe().T.tail(n=3)

In [ ]:
# 타겟 확인 (스케일링 전) > 0/1로 분류
print('스케일링 전 타겟')
data_slim.churned.tail(n=5)

In [ ]:
# 타겟 확인 (스케일링 후) > 큰일 남!
print('스케일링 후 타겟')
data_slim_std_scaled.churned.tail(n=5)

#### 중간분석: 타겟값의 0과 1이 사라진 이유
1. 원인은 StandardScaler 계산식에 있음
    standardized x = (x-mean) / standard deviation
    - 우리 데이터 타겟값의 mean은 0과 1 사이 소숫점 숫자 중 하나임 (전부 0이거나 전부 1이 아닌 한)
    - 각 x에서 이 mean을 빼고 standard deviation으로 나누면 0과 1 이외의 값이 나올 수 있음 (음수 역시 가능)
2. 즉, StandardScaler는 우리 문제에 최적의 스케일러가 아님
3. 수치화 이후 타겟값이 분류에 쓸 수 없는 형태가 되기 때문
4. churned열을 빼고 스케일링을 적용하면 문제 없음
5. 나머지 스케일러 적용 후 결과 확인

- MinMaxScaler (기본 범주인 [0,1] 적용)

In [ ]:
# MinMaxScaler (기본 범주인 (0,1) 적용)
from sklearn.preprocessing import MinMaxScaler

#MMSc = MinMaxScaler(feature_range=(0,3)) # 스케일링 범주에 따른 정확도 차이 확인용
MMSc = MinMaxScaler()
data_slim_MMSc_scaled = pd.DataFrame(MMSc.fit_transform(data_slim), columns = data_slim.columns)

data_slim_MMSc_scaled.describe().T.tail(n=3)

- MaxAbsScaler

In [ ]:
# MaxAbsScaler
from sklearn.preprocessing import MaxAbsScaler

MASc = MaxAbsScaler()
data_slim_MASc_scaled = pd.DataFrame(MASc.fit_transform(data_slim), columns = data_slim.columns)

data_slim_MASc_scaled.describe().T.tail(n=3)

중간분석: MinMax와 MaxAbs가 같은 결과를 내놓는 이유
1. 역시 원인은 계산식에 있음
    - min-max: x_scaled = (x-min(x)) / (max(x)–min(x))
    - max-abs: x_scaled = x / max(abs(x))
2. 두 계산식이 일치하는 경우는 min(x)==0 and max(x)==max(abs(x))
    - 즉 각 열의 값이 0 또는 양수이고 그 최소값이 0일 때
3. 오렌지텔레콤 데이터의 경우 이 사례에 맞음 (아래 참고)
    - 그러므로 MinMax와 MaxAbs 스케일링 후 얻는 결과값이 같음
    - 단, MinMaxScaler의 기본 범주인 [0,1]을 변경하지 않았을 때만 그러함

- 수치화 후, 스케일링 전 데이터 min 확인: 모든 열 >= 0

In [ ]:
# 수치화 후, 스케일링 전 데이터 min 확인: 모든 열 >= 0
pd.DataFrame(data_slim.describe().min(), columns=['min'])

### Intel 방법

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

for col in ['intl_plan', 'voice_mail_plan', 'churned']:
    data[col] = lb.fit_transform(data[col])

In [ ]:
# sklearn 경고 끄기
import warnings
warnings.filterwarnings('ignore', module='sklearn')

from sklearn.preprocessing import MinMaxScaler

msc = MinMaxScaler()

data = pd.DataFrame(msc.fit_transform(data),
                    columns=data.columns)
data.describe().T.tail(n=3)

### 차이와 배움
1. sklearn LabelBinarizer(인텔) vs. pandas factorize(우리)
    - LabelBinarizer는 정수를 / factorize는 소숫자리 값을 줌 (잘못된 내용, 아래 정정사항 참고)
    - 스케일링을 뒤이어 하면 둘 중 뭘 써도 좋음, 스케일링 후엔 모두 소숫자리 값이 됨
    - 스케일링을 하지 않을 거라면 LB가 더 나은 선택, 분류를 표현하기에 더 좋기 때문
    - LabelBinarizer 활용해서 변환해보기 (완료)
2. 인텔: 세 가지 열을 변환하기 위해 for loop 하나를 만들기
    - 가능한 모든 걸 자동화 해보자는 태도가 필요
    - 머리를 바쁘게, 손은 한가롭게
    - 우리도 loop 활용해서 변환 (완료)
3. 적은 게 많은 것
    - dataframe화 하는 단계랑 fit_transform 단계를 한 줄로 합쳐보자 (완료)

[잠깐 실험실] LabelBinarizer로 변환

In [ ]:
# 실험용 데이터 복사
data_lb = data_slim.copy()

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

for column in ['intl_plan', 'voice_mail_plan', 'churned']:
    data_lb[column] = lb.fit_transform(data[column])

data_lb.tail(n=1).T

In [ ]:
# 타겟 변환 여부 상세 확인
data_lb.churned[-5:]

* 실험 종료 (LabelBinarizer로 변환)

#### 참고학습 - 변환 (encoding)
1. 정정사항
    - factorize 역시 정수를 산출 (LabelBinarzier와 같음)
    - 두 변환기 차이의 핵심은 LabelBinarzier의 Binarizer 부분에 있음
        - LabelBinarzier로 카테고리 값을 변환시키는 건 Y/N, T/F 등 선택지가 2(bi)가지일 경우만 가능
        - 더 많은 선택지는 LabelEncorder(한 열이 아닌 인풋 데이터 전체 변환 시, OrdinalEncoder) 사용
        - 이것과 pandas factorize의 기능이 같음
    - 둘 모두 N개의 선택지를 지닌 한 열을 받아 0부터 N-1까지의 숫자로 변환하여 다시 한 열로 돌려줌
        - 위 둘과 차이가 있는 변환기가 바로 pandas의 get_dummies와 sklearn의 OneHotEncoder
        - 이들은 N개의 선택지를 지닌 한 열을 받아 0/1로 구분되는 N(드롭 적용할 경우 N-1)개의 열로 돌려줌
2. Encoder 종류(ordinal/non-ordinal)에 따른 사용 예
    - factorize, LabelEncoder, OrdinalEncoder vs. get_dummies, OneHotEncoder
        - 앞의 셋은 Ordinal 즉 순서(order)가 중요한 분류열 변환에 사용 (예: 차가움/따뜻함/뜨거움)
        - 뒤의 둘은 순서가 상관없는 분류열 변환에 사용 (예: 도베르만/코카스파니엘/시츄)
    - 이유: 인간의 인식처럼 컴퓨터도 큰 숫자에 큰 중요도를 부여함
        - 도베르만/코카스파니엘/시츄를 ordinal encoder로 변환한다면 0/1/2로 바뀜
        - 이는 컴퓨터에게 도베르만 < 코카스파니엘 < 시츄의 중요도 차이를 알려주는 정보가 됨
        - 이를 방지하기 위해 각 종을 T/F로 개별 분류하는 non-ordinal encoder로 변환
3. 관련 문제 확인 - 위에서 state(미국 51개 주 이름) 변환 시 factorize(ordinal encoder)를 사용함
    - 각 주에 의도치 않은 중요도가 부여되었을 수 있음
    - 이런 중요도가 고객 이탈 예측의 정확도에 영향을 끼쳤을 가능성 역시 있음
        - state를 non-ordinal encoder로 변환한 데이터로 다시 실험 (상세 내용은 아래 최종분석 참고)
               - 정확도 차이: 94.34% (ordinal encoding) vs. 91.76% (non-ordinal encoding)

## 문제 3

* 전체 열을 타겟과 트레이닝 데이터로 나누세요. 두 개의 테이블이 나오도록요.
* 위 데이터와 k=3 값을 적용하여 KNN 모델을 학습시키세요. 같은 트레이닝 데이터를 활용해 타겟값을 예측하세요.

### 우리 방법

- 데이터를 X, y로 나누기

X 예시 (전체 중 마지막 세 고객의 마지막 세 열)

In [ ]:
# 위에서 배운 것(drop) 적용
y_MMSc = data_slim_MMSc_scaled['churned'] # 이름 짧게
X_MMSc = data_slim_MMSc_scaled.drop(columns='churned')
X_MMSc.tail(n=3).T.tail(n=3)

y 예시

In [ ]:
print(y_MMSc.tail(n=3).T)

- KNN 학습시키기 / 이탈 여부 예측하기

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=3)
KNN = KNN.fit(X_MMSc, y_MMSc)

y_predict = KNN.predict(X_MMSc)
print('KNN 학습 후 예측값:', y_predict)

# 한 뼘 더: KNN에 탑재된 score 기능을 활용해서 정확도 측정
st_accuracy = KNN.score(X_MMSc, y_MMSc)
print('예측 정확도:', st_accuracy)

#### 중간분석 - 정확도 94.34%의 의미

Q. 학습한 데이터와 예측에 쓰인 데이터가 같지만, 실제값과 다른 예측이 나오는 이유
1. 2주차 학습자료 중 k=1 vs. k=all 비교 그래프를 다시 볼 것 (완료)
2. k=3는 가장 가까운 3개의 다른 사례를 뺀 나머지는 분류에 활용하지 않음을 의미
3. 달리 말해, 예측 고객을 제외한 3개의 다른 고객 사례를 해당 고객의 이탈 여부를 예측하는 데 참고함
4. 이렇게 볼 때 94.34% 예측정확도(실제값과 5.66%p 차이)는 낮지 않음
5. 실제로 KNN에서는 k의 크기가 오버핏/언더핏과 관련이 있음 (아래 최종분석에서 실험)

### Intel 방법

In [ ]:
# 타겟을 제외한 열 목록
x_cols = [x for x in data.columns if x != 'churned']

# 데이터 두 개로 나누기
X_data = data[x_cols]
y_data = data['churned']

# 다른 방법:
# X_data = data.copy()
# y_data = X_data.pop('churned')

- KNN 학습 후 예측값

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn = knn.fit(X_data, y_data)

y_pred = knn.predict(X_data)
y_pred

### 차이와 배움
1. 인텔: 타겟을 뺀 나머지 열을 찾기 위해 for loop 활용
    - 이번에는 우리 방식(drop)이 더 나은 듯함
    - 하지만 인텔의 2번째 방법(pop)이 더 짧고 좋아보임 > 해보자! (아래)
2. 정확도 측정은 다음 문제에서 할 거였는데 미리 해버림. 그럼 아래에서는 직접 기능을 짜보기로 함

[잠깐 실험실] pop으로 데이터 나누기

X 예시 (전체 중 마지막 세 고객의 마지막 세 열)

In [ ]:
popped_X = data.copy()
popped_y = popped_X.pop('churned')
popped_X.tail(n=3).T.tail(n=3)

y 예시

In [ ]:
print(popped_y.tail(n=3))

- 실험 종료 (pop으로 데이터 나누기)

## 문제 4

아직 오류 측정 방법은 안 다뤘죠? 그래도 정확도는 이해하기 쉽습니다. 간단히 전체 중 맞춘 예측의 비율을 계산하는 거니까요.

* 실제 타겟값과 예측값을 활용해 정확도를 측정하는 기능을 만들어보세요.
* 이 기능과 현재 데이터로 직접 만든 KNN 모델의 정확도를 계산하세요.

### 우리 방법

- 정확도 측정 기능 만들기 / 이 측정기로 정확도 예측하기

In [ ]:
# 정확도 측정기
def get_accuracy(prediction, target):
    return sum(prediction == target) / float(len(prediction))

print('우리 정확도:', get_accuracy(y_predict, y_MMSc))

### Intel 방법

In [ ]:
# 맞는 예측 비율 계산기
def accuracy(real, predict):
    return sum(real == predict) / float(real.shape[0])

In [ ]:
print('Intel 정확도:', accuracy(y_data, y_pred))

### 차이와 배움
1. 아 참, 측정 기능을 만들라고 했지 (처음엔 1회성 for loop으로 결과만 얻음)
    - 당장 고치자 (완료)
    - 항상 생각하자 '범용성' '확장성' '재활용성'
2. 94.22% (인텔) vs. 94.34% (우리)
    - 인텔과 우리 모두 같은 스케일링(minmax)과 데이터를 사용
    - 하지만 정확도는 우리 모델이 0.12%p 높음
3. 좀 더 들어가보자

[잠깐 실험실] 0.12% 예측정확도(고객 6명) 차이의 원인

Q1. 정확도 측정기에 차이가 있지 않은가 - 두 데이터(인텔, 우리)를 한 측정기(우리)에 적용해보자

In [ ]:
print('Intel 정확도:', get_accuracy(y_data, y_pred), '   우리 정확도:', get_accuracy(y_MMSc, y_predict))

- 측정기 차이 없음

Q2. 두 예측값 자체가 얼마나 다른가

In [ ]:
print('인텔 예측과 우리 예측의 일치율:', get_accuracy(y_pred, y_predict))

- 어느 정도 다르긴 하지만, 이 한 수치만으론 원인 파악이 힘듦 (인텔/우리 각각의 예측정확도와도 다름)

Q3. 고객이탈(1)로 예측된 값을 모아보자 (총 5000개 사례 중 얼마나 되는지)

In [ ]:
print('인텔 예측:', sum(y_pred), '  우리 예측:', sum(y_predict)) # 예측값
print('인텔 실제:', sum(y_data), '  우리 실제:', sum(y_MMSc)) # 실제값

- 이탈 고객 수 총합만으론 개별 고객 예측값과 실제값의 일치 여부를 알 수 없음

Q4. 각 예측값이 각 실제값과 얼마나 같은가

In [ ]:
print('인텔 예측 일치 고객 수: ', sum(y_pred==y_data), '   우리 예측 일치 고객 수: ', sum(y_predict==y_MMSc))

- 위 수치를 볼 때 인텔보다 우리 모델이 6명 더 많은 고객에 대해 정확한 예측을 내놓는 것을 확인
    - 하지만 여전히 이 차이의 '원인'을 모르겠음

.
.
.
.
.
.
.
.
.
불면과 자기환멸의 밤이 지나고
.
.
.
.
.
.
.
.
.

찾았다! (다음날 낮에 전체 과정을 처음부터 다시 살펴보던 중 원인을 발견)

우리는 트레이닝 데이터에서 'account_length'를 삭제

In [ ]:
print('account_length 유일값 수: ', len(data.account_length.unique()), '   커버리지: ', len(data)/len(data.account_length.unique()))

- 이 열이 가질 수 있는 값의 종류는 218 가지 / 각 값이 갖는 평균 고객의 수는 약 23명

반면 인텔은 'state'를 필요없다고 보고 삭제

In [ ]:
print('state 유일값 수: ', len(data_slim.state.unique()), '   커버리지: ', len(data_slim)/len(data_slim.state.unique()))

- 이 열이 가질 수 있는 값의 종류는 51 가지 / 각 값이 갖는 평균 고객의 수는 약 98명

- 결론 - 원인은 지표 선택에 있었음 (인텔: account_length / 우리: state(ordinal encoded))
- 실험 종료 (예측정확도 차이의 원인)

[잠깐 실험실] state 열을 non-ordinal encoder(get_dummy)로 변환하여 예측정확도 재측정

In [ ]:
# 새 데이터 준비
ohe_st_data = data_raw.copy()
ohe_st_data.drop(['account_length', 'area_code', 'phone_number'], axis=1, inplace=True)
ohe_st_data.tail(n=3).T;

# 각 주별 0/1로 구분하는 열 얻기
st_encoded = pd.get_dummies(ohe_st_data.state, prefix=['state'], drop_first=True)
#print(data_raw.state.nunique(), len(st_encoded.columns)) # 51개 주를 위한 50가지 1/0 구분
st_encoded.describe().T.tail(n=3);

# 인코딩 안 된 다른 열들과 합치기
ohe_st_data = pd.concat([ohe_st_data, st_encoded], axis=1)
#print(len(ohe_st_data.columns))

# 원래 state열 삭제
ohe_st_data.drop(['state'],axis=1, inplace=True)
#print(len(ohe_st_data.columns))
ohe_st_data.tail(n=3).T;

# LB로 Y/N열 변환
lb = LabelBinarizer()
for col in ['intl_plan', 'voice_mail_plan', 'churned']:
    ohe_st_data[col] = lb.fit_transform(ohe_st_data[col])
    
# MinMaxScaler로 스케일링 (기본 범주[0,1] 적용)
MMs = MinMaxScaler()
ohe_st_data_mms = pd.DataFrame(MMs.fit_transform(ohe_st_data), columns = ohe_st_data.columns)
ohe_st_data_mms.describe().T.head(n=3);

# pop으로 X,y 얻기
ohe_st_X = ohe_st_data_mms.copy()
ohe_st_y = ohe_st_X.pop('churned')

# KNN 활용하여 학습 후 정확도 재측정
ohe_st_KNN = KNeighborsClassifier(n_neighbors=3)
ohe_st_KNN = ohe_st_KNN.fit(ohe_st_X, ohe_st_y)
ohe_st_pred = ohe_st_KNN.predict(ohe_st_X)
#print('Predicted Values: ', ohe_st_pred)
ohe_st_accu = ohe_st_KNN.score(ohe_st_X, ohe_st_y)
print('예측정확도:', ohe_st_accu)
print('실제값과 일치하는 예측값의 수: ', sum(ohe_st_pred==ohe_st_y))

- 실험 종료 (state열 non-ordinal 인코딩)

> # - 최종분석: 인텔 모델과 우리 모델의 정확도 차이와 그 원인

- 정확도
        1. 94.22% (예측 일치 고객: 4711명/5000명) - 인텔, account_length 활용
        2. 94.34% (예측 일치 고객: 4717명/5000명) - 우리, state 활용, ordinal encoding
            - 인텔 대비 0.12%p 높은 정확도, 6명 더 많은 고객에 대한 정확한 예측
        3. 99.50% (예측 일치 고객: 4975명/5000명) - 우리, state 활용, non-ordinal encoding (타겟값 포함된 X로 학습 - 최초 버전)
            - 우리 ordinal 대비 5.16%p 높은 정확도, 258명 더 많은 고객에 대한 정확한 예측
            - 인텔 대비 5.28%p 높은 정확도, 264명 더 많은 고객에 대한 정확한 예측
- 정정사항
        1. 91.76% (예측 일치 고객: 4588명/5000명) - 우리, state 활용, non-ordinal encoding (타겟값 없는 X로 학습 - 현재 버전)
            - 우리 ordinal 대비 2.58%p 낮은 정확도, 129명 더 적은 고객에 대한 정확한 예측
            - 인텔 대비 2.46%p 낮은 정확도, 123명 더 적은 고객에 대한 정확한 예측
        2. 오렌지텔레콤에 제공하기 위한 최종 예측 모델 작성 중 오류를 발견
            - 원인: non-ordinal 인코딩 실험 중 일부의 데이터를 X,y로 쪼갤 때 X에서 타겟(churned)열이 분리되지 않음
        3. 이로 인해 모델이 예측해야하는 답을 학습 중 미리 알게 됨 (정확도 상승 요인)
        4. X, y로 데이터 나누는 부분 전수 점검 > X에 타겟열 남아있는 경우 삭제 > 전체 정확도 재측정
            - account_length, area_code(ordinal/non-ordinal), state(ordinal)를 활용한 실험 결과 모두 정확도 변화 없음
            - state(non-ordinal)열 전체 정확도 하락
                - KNN: 99.5% > 91.76% (7.74%p 하락)
                - Logistic Regression 학습제한값 수동 적용(이하 LR): 97.8% > 87.2% (학습되지 않음 - 모든 예측을 0으로 내놓음)
                - LR CrossValidation 학습제한값 적용(이하 LR-CV): 100% > 87.4% (학습했으나 정도가 약함)
        - 과제: 정확도 수정이 필요한 부분에 대한 재분석 (완료)
            - 특히 state와 account_length 비교 / ordinal과 non-ordinal 인코딩 비교 부분

## 1. 예측에 필요한 지표를 선택하는 기준
> 1. 우리는 state보다 account_length가 더 필요치 않은 정보라고 판단하고 삭제
    - 그 결과 인텔 대비 0.12%p(ordinal) 높은 정확도를 얻음
        - 같은 k=3 값, min-max scaler를 적용한 KNN모델 사용
    - 이는 고객 이탈을 예측하는 데 state가 account_length에 비해 0.12%p 더 도움되는 정보를 지니고 있다는 의미
2. 실제로 같은 지역 내 고객들은 생활수준, 브랜드선호도, 혹은 정치성향 등에 있이서 공통점을 보이곤 함
    - 오랜지텔레콤의 통신망을 활용하는 고객 역시 예외는 아닐 것 (통신사를 옮기는 성향도 지역에 반영 가능)
3. 반면 계정의 길이는 이런 성향을 반영하는 지표로 기능한다고 볼 수 없음
    - 고객 계정: 통신사 등록 시 시스템에 의해 자동으로 부여되거나 고객이 입력하여 설정된 문자열
    - 우리 사례의 경우, 계정 길이가 주 이름에 비해 예측에 기여하는 정도가 낮다는 것이 예측정확도 차이로 증명됨

- 결론: 예측 지표로서 state와 account_length 중 하나를 활용해야 한다면 선택은 state가 맞음 (위치 정보를 포함하고 있으므로)
    - ordinal encoding된 state를 KNN 모델로 학습시킨 경우만 그러함
    - state열에 non-ordinal encoding 혹은 Logistic Regression 모델을 적용한 경우 account_length에 비해 낮은 예측정확도를 보임
    - 인코딩 및 학습모델 종류에 따른 정확도 변화 / account_length가 지닌 정보에 대한 심층분석이 아래에서 이어짐

## 2. 위치정보를 포함하는 지표로서 state와 area_code 비교
> 1. 최초 세 열(area_code, account_length, phone_number)을 삭제하는 과정에서 state와 area_code 사이 충분한 고려없이 state를 선택
2. 하지만 이후 state와 account_length의 정확도 비교를 통해 위치정보가 갖는 중요성이 확인됨
3. 보다 효과적인 지표 활용을 위해 위치정보를 포함한 두 열에 대한 1:1 대조가 필요 (아래 실험 참고)

- 결론: area_code가 state보다 효과적인 위치정보 지표 / 하지만 활용 시 100% 예측정확도를 보이므로 오버핏 우려

## 3. ordinal / non-ordinal encoding이 KNN 정확도에 끼치는 영향
> 1. 결과: state 열에 non-ordinal encoding 적용 후 정확도 하락
    - 2.58%p (94.34%-91.76%) / 고객 수 기준: 129명 (4717명-4588명)
2. 원인: non-ordinal encoding을 통해 50개 열 추가 (기존 state열 제외 나머지 열의 수: 16개)
    - state열 하나의 정보를 위해 다른 모든 열을 합친 것의 3배가 넘는 열이 할당
    - 그만큼 다른 지표의 중요도가 낮아짐 (하나의 열이 전체에서 차지하는 비중이 줄어들었으므로)
    - 이 역효과의 크기가 열 종류에 맞는 인코딩으로 기대되는 정확도 상승폭보다 큰 것으로 보임
3. 의문: state열에서 이런 정확도 변화를 보였다면, area_code열은 어떠한가? (현재 변환없이 scaling만 적용)
    - 공통점: area_code도 state와 마찬가지로 서로 독립적인 의미를 지니는 별개의 값 (숫자지만 수치적 의미를 지니지 않음)
    - 차이점: area_code의 유일값은 세 종류로 non-ordinal encoding을 적용해도 1개 열 추가에 그침 (state의 50개 열 추가와 대조)
    - 과제: area_code에 non-ordinal encoding 적용한 버전으로 실험 (추가분석 참고)

- 결론: non-ordinal encoding으로 변환되는 열의 유일값 종류가 전체 열의 수보다 많다면 인코딩으로 인한 다른 열의 중요도 하락 주의

[잠깐 실험실] state / area_code 1:1 대조

In [ ]:
# 각 지표가 갖는 유일값 확인 (처음 저장해둔 원형 데이터 활용)
print('area code 유일값:', data_raw.area_code.unique())
print('state 유일값:', data_raw.state.unique())

# 두 수치 비교
area_coverage, state_coverage = len(data_raw)/len(data_raw.area_code.unique()), len(data_raw)/len(data_raw.state.unique())
print('두 지표의 각 값이 포함하는 고객 수 비율: ', area_coverage/state_coverage)

1. 결과
    - area_code가 가질 수 있는 값의 종류 = 3
    - state가 가질 수 있는 값 종류 = 51
    - area_code 커버리지(한 값이 품을 수 있는 평균 고객 수) : state 커버리지 = 17 : 1
2. 의문
    - area_code의 높은 커버리지가 예측정확도에 (얼마나) 기여하는가

- state를 제외하고 area_code를 활용한 데이터로 정확도 확인

In [ ]:
# 1. 필요없는 열 삭제
ac_col_unused = ['state', 'account_length', 'phone_number']
ac_data = data_raw.drop(columns=ac_col_unused)
#print(len(ac_data.columns)) # 18 = 21 - 3
ac_data.tail(n=1);

# 2. LabelBinarizer로 변환
lb = LabelBinarizer()
for i in range(len(ac_data.columns)):
    if ac_data.dtypes[i] != int and ac_data.dtypes[i] != float: # 숫자 여부 확인
        col = ac_data.columns[i]
        ac_data[col]= lb.fit_transform(ac_data[column])
ac_data.tail(n=1).T;

# 3. MinMaxScaler로 스케일링 (기본 범주[0,1] 적용)
MMs = MinMaxScaler()
ac_data_MMs = pd.DataFrame(MMs.fit_transform(ac_data), columns = ac_data.columns)
ac_data_MMs.describe().T.head(n=3);

# 4. pop으로 X,y 얻기
ac_X = ac_data_MMs.copy()
ac_y = ac_X.pop('churned')
#print(ac_y.tail(n=3))
ac_X.tail(n=3).T.tail(n=3);

# 5. KNN 학습 / 이탈 여부 예측
ac_KNN = KNeighborsClassifier(n_neighbors=3)
ac_KNN = ac_KNN.fit(ac_X, ac_y)
ac_pred = ac_KNN.predict(ac_X)
#print(ac_pred)
ac_accu_k3 = ac_KNN.score(ac_X, ac_y)
print('예측정확도:', ac_accu_k3)

1. 결과
    - state를 area_code로 교체한 뒤 같은 KNN으로 얻은 정확도 변화: 94.34% > 100.00% (5.66%p 상승)
2. 의문
    - 우리 정확도 측정기에 문제가 있는 것은 아닌가
        - 세 가지 정확도 측정기 (우리 / 인텔 / KNN.score)로 다시 확인

In [ ]:
# 세 가지 측정기로 area_code 정확도 재확인

ac_accu_my = get_accuracy(ac_y, ac_pred)
ac_accu_intel = accuracy(ac_y, ac_pred)
ac_accu_given = ac_KNN.score(ac_X, ac_y)

print('우리 측정기: ', ac_accu_my)
print('인텔 측정기: ', ac_accu_intel)
print('KNN.score: ', ac_accu_given)
# 실제값/예측값 일치 정도
print('실제값과 일치하는 예측값의 수:', sum(ac_y==ac_pred))

- 실험 종료 (state / area_code 1:1 대조)

1. 결과
    - 세 측정기 결과값 같음
    - 실제값과 같은 예측값의 비율(1 = 5,000/5,000)과도 일치
    - 우리 측정기 문제 없음
2. 의문
    - 측정기에 문제가 없이, 100%의 정확도가 나왔다면
    - area_code와 고객이탈율 사이의 상관관계가 너무 높은 것은 아닌가
    - 기존 16개 지표에 area_code가 추가됨으로써 고객이탈율 예측에 충분한 조합이 구성된 것은 아닌가 (오버핏)
    - KNN의 k값을 늘려가며(학습제한을 강화해가며) 정확도 변화를 파악

[잠깐 실험실] KNN모델의 k값 증가에 따른 예측정확도 변화

In [ ]:
# k값 증가에 따른 정확도 기록기 (우리 측정기 활용)
def accu_lister(X, y, k):
    list = []
    for i in range(k, k+50): 
        KNN = KNeighborsClassifier(n_neighbors=i+1)
        KNN = KNN.fit(X, y)
        pred = KNN.predict(X)
        list.append(get_accuracy(y, pred))
    return list
k = 1380
accu_list = accu_lister(ac_X, ac_y, k);

# K값 증가에 따른 정확도 변화 그래프
import matplotlib.pyplot as plt
plt.figure(figsize=(7, 5))
plt.style.use('seaborn-dark')
plt.xlabel('K')
plt.ylabel('Accuracy')
plt.title('Change of Accuracy by Increase of K')
plt.plot(np.arange(k, k+50, step=1), accu_list, color='k', alpha=0.73, linewidth=7.3);

- K=1420 이후 정확도 86%가 갖는 의미

In [ ]:
ac_KNN = KNeighborsClassifier(n_neighbors=1420)
ac_KNN = ac_KNN.fit(ac_X, ac_y)

ac_pred = ac_KNN.predict(ac_X)
ac_accu = ac_KNN.score(ac_X, ac_y)
print('k=1420일 때 예측정확도:', ac_accu)
#print(ac_pred)
print('모든 (이탈)예측의 합:', sum(ac_pred))

- 실험 종료 (k값 증가에 따른 예측정확도 변화)

1. 결과
    - 최초 실험 범위 k = [1:5000(N, 고객 수)] > 너무 긴 러닝타임 > [1:1500] 시도 > 1400 부근 50 범위에서 유일한 변화 확인
    - 50 범위로 실험할 수 있도록 기능 변경 (완료)
    - 1380 부근까지 예측정확도 100%
    - 1420 부근부터 예측정확도 86%
        - 전체 중 실제 비이탈(0) 고객의 비율이 86%
        - 모든 예측을 0으로 내놓았기 때문에 나온 수치
        - 이 지점보다 큰 k값으로는 학습이 이루어지지 않음을 의미
2. 의문
    - 1380이면 전체 고객(5000)의 약 1/4에 해당하는 숫자
    - 이 정도의 K값을 주어야만 억제되는 학습이라면 오버핏으로 봐야하지 않는가
        - 즉, KNN모델을 사용하여 고객이탈율을 예측하기에는 area_code가 지표로서 부적합하지 않은가
    - 오버핏 검증을 위해 데이터를 트레이닝셋과 타겟셋으로 나누는 방법은 어떤가
        - KNN은 트레이닝 때 데이터의 값만 저장해두고 실제론 예측에서 트레이닝에 해당하는 학습을 하기 때문에 적용 불가
    - 다른 모델은 어떤가 (딥러닝 등)
        - 우선 다른 머신러닝 모델을 적용해본 뒤 판단

> # - 추가분석: KNN 외 다른 모델 적용

### Logistic Regression

- 지표별(area_code, state, account_length) 새 데이터 준비 및 전처리

In [ ]:
# 트레이닝셋/테스트셋으로 데이터 나누기
def data_split(X, y, s, r): # s=전체 데이터 중 테스트셋 비율 / r=랜덤시드
    from sklearn.model_selection import train_test_split
    len_list = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=s, random_state=r)
    for i in [X_train, X_test, y_train, y_test]:
        len_list.append(len(i))
    return X_train, X_test, y_train, y_test, len_list
ac_X_train, ac_X_test, ac_y_train, ac_y_test, ac_len_list = data_split(ac_X, ac_y, 0.3, 13) # area_code
st_X_train, st_X_test, st_y_train, st_y_test, st_len_list = data_split(X_MMSc, y_MMSc, 0.3, 13) # state
al_X_train, al_X_test, al_y_train, al_y_test, al_len_list = data_split(X_data, y_data, 0.3, 13); # account_length

- area code 트레이닝셋 - X 예시 (전체 중 마지막 세 고객의 마지막 세 열)

In [ ]:
# 이 중 area_code 트레이닝셋 확인
ac_X_train.head(n=3).T.head(n=3)

- area code 테스트셋 - X 예시 (전체 중 마지막 세 고객의 마지막 세 열)

In [ ]:
# 이 중 area_code 테스트셋 확인
ac_X_test.head(n=3).T.head(n=3)

- Logistic Regression 학습 결과 (penalty = L2, 학습제한값 C = 0.01 수동 적용)

In [ ]:
# logistic regression으로 학습
def lr_fit(X_train, y_train, X_test, y_test, penalty, C):
    from sklearn.linear_model import LogisticRegression
    LR= LogisticRegression(penalty=penalty, C=C) # penalty = 학습제한 방법 / C = 제한 정도 (클수록 덜 제한)
    LR= LR.fit(X_train, y_train)
    predict = LR.predict(X_test)
    accuracy = get_accuracy(y_test, predict)
    return predict, accuracy

lr_ac_pred, lr_ac_accu = lr_fit(ac_X_train, ac_y_train, ac_X_test, ac_y_test, 'l2', 0.01)
lr_st_pred, lr_st_accu = lr_fit(st_X_train, st_y_train, st_X_test, st_y_test, 'l2', 0.01)
lr_al_pred, lr_al_accu = lr_fit(al_X_train, al_y_train, al_X_test, al_y_test, 'l2', 0.01)
result = {'Area Code': [lr_ac_accu, sum(lr_ac_pred)],
          'State': [lr_st_accu, sum(lr_st_pred)],
         'Account Length': [lr_al_accu, sum(lr_al_pred)]}
result = pd.DataFrame(result, index=['Accuracy', 'Sum of predictions'])
result

#### 중간분석 - area_code의 예측 기여도 확인
1. logistic regression을 적용 후 area_code만 100% 정확한 예측을 보임
2. 같은 조건으로 state, account_length는 모든 예측을 0으로 내놓음 (학습되지 않음)
3. 학습제한값(C)이 0.1 이상으로 커지면(제한이 덜해지면) state, account_length도 100% 예측을 보임

- Logistic Regression 학습 결과 (LogisticRegressionCV로 얻은 학습제한값 C 적용)

In [ ]:
# Logistic Rregression에 탑재된 cross validation으로 최적값 학습 후 다시 예측
def LR_cross_valid(X_train, y_train, X_test, y_test, f, r): # f = 트레이닝 데이터 분리 수 / r = 랜덤시드
    from sklearn.linear_model import LogisticRegressionCV
    LR_CV = LogisticRegressionCV(cv=f, random_state=r).fit(X_train, y_train)
    LR_CV.predict(X_test)
    prediction = LR_CV.predict(X_test)
    accuracy = get_accuracy(prediction, y_test)
    return prediction, accuracy

lr_ac_pred_cv, lr_ac_accu_cv = LR_cross_valid(ac_X_train, ac_y_train, ac_X_test, ac_y_test, 5, 13)
lr_st_pred_cv, lr_st_accu_cv = LR_cross_valid(st_X_train, st_y_train, st_X_test, st_y_test, 5, 13)
lr_al_pred_cv, lr_al_accu_cv = LR_cross_valid(al_X_train, al_y_train, al_X_test, al_y_test, 5, 13)
result_cv = {'Area Code': [lr_ac_accu_cv, sum(lr_ac_pred_cv)],
          'State': [lr_st_accu_cv, sum(lr_st_pred_cv)],
         'Account Length': [lr_al_accu_cv, sum(lr_al_pred_cv)]}
#print('Results from LR (with parameters picked by LogisticRegressionCV)')
result_cv = pd.DataFrame(result_cv, index=['Accuracy', 'Sum of predictions'])
result_cv

#### 중간분석 - area_code의 예측 기여도 재확인
1. Logistic regression의 cross valiation으로 각 지표에 최적화된 C(제한값)를 찾아 학습에 적용
    - 여전히 area_code를 활용한 데이터의 정확도가 가장 높음 (100%)
2. 다른 두 지표로 얻은 정확도는 최초 실험 (C=0.01로 수동 설정) 때보다 상승 (모든 예측을 0으로 내놓는 현상 중지)
    - CV에서 실험되는 C값(10부터 시작: 숫자가 클수록 제한이 덜함)이 0.01까지 내려가진 않은 것으로 보임
        - 어느 시점부터 모든 예측이 0이 되므로 그 전에 최적값 설정      
3. 87.67% (state, ordinal encoding) vs. 87.8% (account_length)
    - 적합하지 않은 인코딩으로 state열이 변환된 경우 account_length열을 활용할 때보다 예측정확도가 낮은 것으로 보임
        - state에 non-ordinal encoding 적용 후 결과 대조

[잠깐 실험실] ordinal/nonordinal 인코딩이 Logistic Regression 정확도에 끼치는 영향

In [ ]:
# encoding된 state 데이터셋 나누기
ohe_st_X_train, ohe_st_X_test, ohe_st_y_train, ohe_st_y_test, ohe_st_len_list = data_split(ohe_st_X, ohe_st_y, 0.3, 13) # state 활용

# 학습제한값 수동 적용한 Logistic Regression으로 학습
ohe_st_pred_lr, ohe_st_accu_lr = lr_fit(ohe_st_X_train, ohe_st_y_train, ohe_st_X_test, ohe_st_y_test, 'l2', 0.01)

# Cross Validation으로 얻은 학습제한값 적용한 Logistic Regression으로 학습
ohe_st_pred_cv, ohe_st_accu_cv = LR_cross_valid(ohe_st_X_train, ohe_st_y_train, ohe_st_X_test, ohe_st_y_test, 5, 13)

print('수동 C 적용 Logistic Regression 예측정확도(state, non-ordinal):', ohe_st_accu_lr)
print('CV C 적용 Logistic Regression 예측정확도(state, non-ordinal):', ohe_st_accu_cv)

결과분석
1. 정확도 차이 (state열 활용 데이터)
    - 87.2% (ordinal, 수동 제한값) : 87.2% (non-ordinal, 수동 제한값)
    - 87.67% (ordinal, CV 제한값) : 87.4% (non-ordinal, CV 제한값)
    - 다른 열과의 비교
        - 87.8% (account_length열, CV 제한값) : 87.4% (state열, non-ordinal, CV 제한값)
2. 의미
    - Logistic Regression 모델도 non-ordinal encoding 적용 시 예측정확도 소폭 하락
    - 하락의 정도는 KNN 모델에 비해 덜함
        - KNN: 2.58%p 하락
        - LR(CV 제한값): 0.27%p 하락
        - LR(수동 제한값): 변화 없음 - 두 모델 다 모든 예측을 0으로 내놓음
        - 원인 추론
            - 테스트셋을 전부 학습에 활용하는 KNN의 특징: 오버핏 가능성 높음 > 외부 변화에 취약 > 더 큰 하락
            - LR에 주어지는 학습제한값: 오버핏 억제 > 일반화 가능성 높음 > 하락폭 억제
3. 결론
    - KNN, LR 모델 모두 state에 non-ordinal 인코딩 적용 시 정확도 하락 (0.27%p ~ 2.58%p)
    - 인코딩 전, 변환 후 추가될 열들과 다른 열들의 중요도 비교 필요
    - 추가되는 열이 기존 열에 비해 중요도가 낮고 그 수가 기존 열보다 많다면 (다른 열의 중요도를 흐린다면)
        - 아예 해당 열을 사용하지 않거나 (학습에 방해가 되므로)
            - 인텔의 선택
        - 적합하지 않은 종류의 인코딩으로 얻을 수 있는 정확도의 한계를 감안하고 ordinal 인코딩 적용 (열이 추가되지 않으므로)
            - 우리의 선택

- 실험 종료 (ordinal/nonordinal 인코딩이 Logistic Regression 정확도에 끼치는 영향)

### 추가의문: 다른 실험이 필요한가? 그렇다면 어떤 모델을 적용할 것인가?
1. 우리의 최초 의문은 풀렸는가?
    - 인텔 모델과 우리 모델(state, ordinal, KNN)의 정확도 0.12%p 차이의 원인은 무엇인가?
        - 인텔은 account_length를 / 우리는 state를 지표로 활용 (다른 지표 동일)
        - state에 포함된 위치 정보가 account_length에 포함된 정보에 비해 고객이탈율 예측에 더 효과적
            - KNN, Logistic Regression 두 모델을 활용한 실험 각각에서 검증
           >- 정정사항
                1. Logistic Regression의 경우 account_length 대비 state(non-ordinal) 정확도 낮음 (0.13%p)
                2. state열이 account_length열에 비해 절대적으로 도움되는 정보를 지니진 않음
                3. 인코딩 방식과 모델의 종류에 따라 상대적 중요성이 달라질 수 있음
                4. 하지만 이로 인한 둘 사이 정확도 차이가 다른 지표(area_code)와의 차이에 비해 크지 않음
    - 위치 정보가 예측에 중요한 지표라면 state와 area_code 중 무엇이 더 효과적인가?
        - area_code가 더 효과적인 지표
            - KNN 모델로 예측 시, 정확도 94.34% (state) vs. 100% (area_code)
            - Logistic Regression(수동 학습제한값 0.01) 적용 시, 87.2% (state) vs. 100% (area_code)
    - area_code로 인해 100% 예측이 가능하다는 건 오버핏의 증거 아닌가?
        - 트레이닝셋과 테스트셋으로 데이터가 구분된 Logistic Regression으로 예측 시
            - 트레이닝 중 보지 못한 테스트셋에 100% 정확도를 보임 (state / account_length는 그렇지 않음)
        - 테스트셋 비율 증가에 따른 정확도 변화 실험 후 (area_code, non-ordinal encoding, LR-CV)
            - 테스트셋 비율 10%부터 90%까지 예측 정확도 100% 유지 (아래 실험 참고)
2. 다른 머신러닝 모델로 위 결과에 대한 확인이 더 필요한가?
    - KNN / Logistic Regression과 학습의 형태가 다른가? (다른 결과가 기대되는가?)
3. 딥러닝 모델을 적용하는 게 맞는가?
    - 5,000개의 인풋 / 17개의 지표 > 딥러닝 모델을 적용할만한 크기와 복잡성을 지닌 데이터인가? (오버핏이 우려되지 않는가?)
4. 판단에 앞서, state와 마찬가지로 area_code열에 non-ordinal encoding 적용 후 정확도 변화 확인

[잠깐 실험실] area_code에 non-ordinal encdoing 적용

In [ ]:
# scaling 전 area_code 포함된 데이터 불러오기
ohe_ac_data = ac_data.copy()

# 각 area_code별 0/1로 구분하는 열 얻기
ac_encoded = pd.get_dummies(ohe_ac_data.area_code, prefix=['area_code'], drop_first=True)
#print(data_raw.area_code.nunique(), len(ac_encoded.columns)) # 3개 area_code를 위한 2가지 1/0 구분
ac_encoded.describe().T.tail(n=3);

# 인코딩 안 된 다른 열들과 합치기
ohe_ac_data = pd.concat([ohe_ac_data, ac_encoded], axis=1)
#print(len(ohe_ac_data.columns)) # 20 = 18 + 2

# 원래 area_code열 삭제
ohe_ac_data.drop(['area_code'],axis=1, inplace=True)
#print(len(ohe_ac_data.columns)) # 19 = 20 - 1
ohe_ac_data.tail(n=3).T;

# MinMaxScaler로 스케일링 (기본 범주[0,1] 적용)
MMs = MinMaxScaler()
ohe_ac_data_mms = pd.DataFrame(MMs.fit_transform(ohe_ac_data), columns = ohe_ac_data.columns)
ohe_ac_data_mms.describe().T.tail(n=3);

# pop으로 X,y 얻기
ohe_ac_X = ohe_ac_data_mms.copy()
ohe_ac_y = ohe_ac_X.pop('churned')

# KNN 활용하여 학습 후 정확도 재측정
ohe_ac_KNN = KNeighborsClassifier(n_neighbors=3)
ohe_ac_KNN = ohe_ac_KNN.fit(ohe_ac_X, ohe_ac_y)
ohe_ac_pred = ohe_ac_KNN.predict(ohe_ac_X)
#print(ohe_ac_pred)
#print(sum(ohe_ac_pred==ohe_ac_y))
ohe_ac_accu = ohe_ac_KNN.score(ohe_ac_X, ohe_ac_y)
print('KNN 예측정확도:',ohe_ac_accu)


# Logistic Regression 활용하여 학습 후 정확도 재측정
# 트레이닝셋 / 테스트셋 나누기
ohe_ac_X_train, ohe_ac_X_test, ohe_ac_y_train, ohe_ac_y_test, _ = data_split(ohe_ac_X, ohe_ac_y, 0.3, 13);
# 학습제한값 수동 적용 logistic regression으로 학습
ohe_ac_pred_lr, ohe_ac_accu_lr = lr_fit(ohe_ac_X_train, ohe_ac_y_train, ohe_ac_X_test, ohe_ac_y_test, 'l2', 0.01)
# Cross validation으로 학습된 제한값 적용 logistic regression으로 학습
ohe_ac_pred_cv, ohe_ac_accu_cv = LR_cross_valid(ohe_ac_X_train, ohe_ac_y_train, ohe_ac_X_test, ohe_ac_y_test, 5, 13)
print('LR 예측정확도:',ohe_ac_accu_lr)
print('LR_CV 예측정확도:',ohe_ac_accu_cv)

- 실험 종료 (area_code에 non-ordinal encdoing 적용)

#### 결과분석 (area_code 열을 non-ordinal encoding으로 변환한 결과)
1. KNN: 100% (ordianl encoding 대비 변화 없음)
2. Logistic Regression (수동 제한값): 100% (ordianl encoding 대비 변화 없음)
3. Logistic Regression (CV 제한값): 100% (ordianl encoding 대비 변화 없음)

## 지표별 / 인코딩별 / 모델별 예측정확도 총정리

In [ ]:
import pandas as pd
accuracies = pd.DataFrame({'feature': ['account_length', 'state', 'state', 'area_code', 'area_code'], 
                          'encoding': ['not applicable', 'ordinal', 'non-ordinal', 'ordinal', 'non-ordinal'],
                          'KNN': [0.9422, 0.9434, ohe_st_accu, ac_accu_k3, ohe_ac_accu], 
                          'LR': [lr_al_accu, lr_st_accu, ohe_st_accu_lr, lr_ac_accu, ohe_ac_accu_lr],
                          'LR-CV': [lr_al_accu_cv, lr_st_accu_cv, ohe_st_accu_cv, lr_ac_accu_cv, ohe_ac_accu_cv]},
                          columns=['feature', 'encoding', 'KNN', 'LR', 'LR-CV'])
#accuracies.set_index(['feature', 'encoding'])
accuracies

area_code 활용 시 지속적으로 100% 정확도를 얻는 이유 추론
    1. 데이터 내 총 세 가지 area_code값이 존재: 408 415 510
        - 408: California 내의 두 지역을 커버 (주로 San Jose)
        - 415: 위와 동일 (주로 San Francisco)
        - 510: 위와 동일 (주로 Oakland)
    2. 데이터 수집 기간 내, 이 중 특정 지역에서 고객 이탈이 더 많이 일어났을 수 있음 (해당 지역 통신망 고장 등으로)
        - 해당 지역 고객에 대한 이탈 예측 편중 > 오버핏
        - 각 area_code에 따른 고객이탈율 확인 필요

[잠깐 실험실] 세 area_code의 고객이탈율 차이

In [ ]:
# 전체 예측 중 이탈 고객 비율
print('전체 고객 중 이탈 고객의 비율:', sum(y_data)/len(y_data))

area_code별 이탈 고객의 비율 (마지막 열)

In [ ]:
# area_code 열과 해당 고객이탈 열만으로 구성된 데이터 준비
data_ac_only = pd.DataFrame(data_raw.area_code).join(y_data)

# 각 area_code의 총 이탈 고객 수 데이터에 추가
data_ac_only_sum = data_ac_only.groupby('area_code')['churned'].sum().to_frame().reset_index()

# 각 area_code의 총 고객 수 데이터에 추가
data_ac_only_sum['total'] = data_raw.area_code.value_counts().tolist()

# 각 area_code의 총 고객 중 이탈한 고객의 비율 데이터에 추가
data_ac_only_sum['propotion']=data_ac_only_sum.churned/data_ac_only_sum.total
data_ac_only_sum

- 실험 종료 (세 area_code의 고객이탈율 차이)

결과분석
1. 세 area_code 사이 이탈율 차이가 있으나 전체 예측을 압도하는 정도로 한 지역에 쏠려있진 않음
    - 408지역 : 415지역 : 510지역 = 약 1 : 4 : 2의 비율
    - 전체 이탈율(14.14%) 대비 약 0.5: 2: 1의 비율
    - 즉 area_code의 활용으로 인한 정확도 상승이 특정 지역의 예외적 이탈율 때문이라고 볼 수 없음
    
의문
1. area_code 단독이 아닌 다른 지표들과의 결합으로 인한 효과는 없는가?
    - 있다면 어떻게 측정/검증할 것인가?
        - 의문2에 대한 실험 후 account_length가 계약 유지기간이 아니라는 확신이 있을 경우 진행
2. 보다 근본적으로, 인텔은 왜 account_length를 선택하였나? (많이 늦었지만 언젠간 해야하는 질문)
    - 우리가 기존에 알고 있던 (그리고 검색을 통해 확인한) '계정 길이'와는 다른 정보를 이 열이 지니진 않는가?
    - 만약 이 열의 length가 흔히 말하는 고객의 통신사 계약 유지기간이라면 이는 예측에 효과적인 지표가 아닌가?
    - 현재 가지고 있는 정보(account_length 값)만으로 위 가설을 검증할 수 있는가?

[잠깐 실험실] account_length 상세점검

In [ ]:
# 기본 수치 확인 - 218개 유일값 / 평균 100 / 1부터 243까지
print('acount_length 유일값의 수:', data_raw.account_length.nunique())

기본 통계수치

In [ ]:
data_raw.account_length.describe()

히스토그램

In [ ]:
# 빈도수 확인
plt.figure(figsize=(7, 5))
plt.style.use('seaborn-dark')
plt.xlabel('Account Length')
plt.ylabel('Frequency')
plt.title('Number of Customers by Account Length')
plt.hist(data_raw.account_length, color='darkcyan');

- 실험 종료 (account_length 상세점검)

결과분석
1. 전체 고객: 5000명/ 유일값: 218개 / 값의 범위: 1 ~ 243
2. 73 ~ 127 구간에 50%의 고객이 속함
3. 평균값인 100을 중심으로 정규분포을 보이지만 200 이상의 꼬리 구간도 존재

의문
1. 계정(고객ID) 길이일 가능성
    - 길이의 대상은 무엇인가? (유니코드, 2진법 숫자열 등)
    - 경험적으로, 1자리 길이의 계정 설정이 가능한가? (유니코드의 경우)
    - 243자리 길이의 계정 역시 가능한가?
2. 계약 유지기간일 가능성
    - 기간의 단위는 무엇인가? (일, 주, 월, 혹은 그 외 다른 단위)
    - 단위가 '주'라면 위 히스토그램을 어떻게 해석할 수 있는가?
        - 5000명 고객들은 평균적으로 100주(약 2년) 동안 계약 유지
        - 그 중 50%는 73주(약 1년 반) ~ 127주(약 2년 반) 동안 계약 유지
        - 200주(약 4년) 이상 유지한 고객 역시 소수지만 존재
3. 제3의 가능성
    - 다른 지표를 이름만 account_length로 바꾼 것 아닌가? (개인정보보호 등의 이유로)
        - 그렇다면 이를 알아낼 방법이 있는가?
4. 다른 근본적인 의문
    - account_length가 계약 유지기간이고 그만큼 예측에 효과적인 지표로 (경험적으로) 기대된다면,
        - 왜 예측정확도는 state, area_code를 활용했을 때보다 낮은가?

[잠깐 실험실] 다른 데이터의 계약 유지기간 지표와 대조

1. 데이터: 통신회사 Telco의 고객 이탈율 예측을 위한 자료 ([IBM이 Kaggle에 학습용 자료로 제공](https://www.kaggle.com/blastchar/telco-customer-churn))
2. 공통점: 예측목표 (고객 이탈 여부) / 주어진 지표의 상당 수 (사용한 요금, 부가서비스 가입 여부 등)
3. 차이점:
    - 성별 / 나이 / 가족 등 우리 예측에서는 고려되지 않던 고객 개인정보를 지표로 제공
    - 인터넷서비스, 스트리밍비디오 등 기본 전화 기능 외 부가서비스에 관한 지표 제공
    - 고객의 계약 유지기간을 tenure란 이름의 지표로 명시하여 제공
        - 반면 account length와 관련된 지표 없음
        - 고객 아이디가 제공되지만 인덱스로 1:1 매칭 가능
            - 즉 지표로서 기능하지 않음
4. Telco의 tenure열(단위: 월)과 우리의 account_length열 상세 비교
    - account_length 단위: '주'로 가정 후 tenure에 맞추어 월로 변환

통계수치 1:1 비교/대조

In [ ]:
# 데이터 불러오기
data_telco = pd.read_csv('../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 1:1 대조 (account_length 단위 월(30일로 가정)로 변경)
comparison = pd.DataFrame(data_telco.tenure.describe()).join(pd.DataFrame(data_raw.account_length.describe()))
comparison.iloc[1:,1] = comparison.iloc[1:,1]/4.5
comparison

히스토그램

In [ ]:
# 히스토그램
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12.5, 5))

plt.subplot(1, 2, 1)
plt.hist(data_telco.tenure, color='darkcyan', alpha=0.37)
plt.title('Tenure (Telco)')

plt.subplot(1, 2, 2)
plt.hist(data_raw.account_length/4.5, color='darkcyan')
plt.title('Account Length (Orange Telecom)')

fig.tight_layout()

- 결과분석
    1. 기간 - account_length의 단위를 '주'로 가정했을 경우, 전체 구간의 길이가 유사해짐
    2. 분포 형태 - tenure와 정확히 반대
        - 같은 통신업계 / 5,000~7,000명의 비슷한 고객 수 /  둘 다 학습을 위해 제공된 데이터
            - 두 지표가 같은 값을 나타낼 확률이 낮음 (둘 중 하나 이상이 특이사례가 아니라면)
    3. 최초 결론 - 우리 데이터의 account_length 지표는 계약 유지기간일 가능성이 낮음 (아래에서 반론)

- 실험 종료 (다른 데이터의 계약 유지기간 지표와 대조)

### 남은 의문들
1. 오렌지텔레콤과 텔코의 사업영역이 (혹은 데이터가 수집된 시대가) 다른 것은 아닌가?
    - 같은 통신업계라도 집전화 사업과 휴대폰(특히 스마트폰) 사업의 모델과 고객 구조는 다를 수 있음
    - 텔코의 지표엔 있으나 오렌지텔레콤엔 없는 지표 중 계약 형태가 있음 (매월, 1년, 2년 등)
    - 이런 계약의 형태에서 오는 차이가 고객 이탈율 분포의 차이를 만들어낸 것이 아닌가?
2. 머신러닝 모델(KNN, Logistic Regression)만으로도 100% 정확도를 보이는 상황에서 딥러닝 모델을 적용해야 하는가?
    - 적용한다면 모든 지표를 활용하여, 모델이 직접 가중치 설정을 통해 적정 지표를 찾도록 할 것인가?
    - 아니면 우리 실험을 통해 검증된 지표(area_code와 다른 16개 지표)만 제공하여 학습 효율을 높일 것인가?
    - 딥러닝 모델에서도 (테스트셋에 대해) 100% 정확도가 나온다면
        - 이것 역시 오버핏 때문이 아니라는 가설을 어떻게 검증할 것인가?
3. 만약 오늘, 최종 고객이탈예측 모델을 오렌지텔레콤에 제공해야 한다면
    - 무엇을 포기할 수 없는가?
        - 예측 정확도 - 향후 오렌지텔레콤 담당자가 활용할 지표로 끌어낼 수 있는 최적의 예측값 (완료)
        - 사용 편의성 - 데이터 전문가가 아니라도 쉽게 값을 입력하고 결과를 확인할 수 있도록 쉬운 사용성 (완료)
        - 선제적 대안 - 이탈 예상 고객에게 취할 수 있는 이탈 방지 대책 (요금할인, 부가서비스, 안부문자 등) (진행 중)
    - 위의 포기할 수 없는 항목들을 모아 실제 이용가능한 상품으로 만들기까지 얼마의 시간과 자원이 필요한가?
        - 현재 100% 정확도를 보이는 지표/모델 조합에 대한 최종 검증: 2~3시간 예측 (실제 2시간 소요)
        - 검증 후 최종 선정된 모델의 사용성 파악 및 개선: 1~2시간 예측 (실제 4시간 이상 소요)
        - 이탈 방지 대안의 비용 대비 효과 예측: 모델 검증/사용성 개선 이후 모든 시간 (진행 중)
            - 고객 이탈 예측 모델에 필수적인 두 항목(예측 정확도, 사용 편의성)을 먼저 해결 후
            - 고객 가치와 이탈 가능성에 따른 대안의 시급성 별 / 대안 종류 별 경우의 수를 고려한 이탈 방지 전략 모델 개발
    - 2020년 1월 4일 19시 26분 기준, 고객 이탈 예측 모델 완성하여 전달
        - 이탈 방지 모델은 부가기능의 형태로 추후 제공

[잠깐 실험실] area_code열 활용으로 인한 100% 정확도의 오버핏 여부 확인
- 전체 데이터 중 테스트셋 비율에 따른 예측정확도 변화 (area_code열 활용)

In [ ]:
# 테스트셋 비율에 따른 정확도 측정기
def ratio_acc(ratio_X, ratio_y, ratio_list):
    lr_accu_list = []
    cv_accu_list = []
    ratio_check = []
    for ratio in ratio_list:
        ratio_X_train, ratio_X_test, ratio_y_train,  ratio_y_test, _ = data_split(ratio_X, ratio_y, ratio, 13) # 랜덤시드=13
        ratio_pred_lr, ratio_accu_lr = lr_fit(ratio_X_train, ratio_y_train, ratio_X_test, ratio_y_test, 'l2', 0.01)
        ratio_pred_cv, ratio_accu_cv = LR_cross_valid(ratio_X_train, ratio_y_train, ratio_X_test,  ratio_y_test, 5, 13) # cross validation용 데이터 분리 수=5, 랜덤시드 =13
        lr_accu_list.append(ratio_accu_lr)
        cv_accu_list.append(ratio_accu_cv)
        ratio_check.append(ratio)
    return lr_accu_list, cv_accu_list, ratio_check

# 두 버전의 logistic regression(수동 제한값 / CV 제한값)으로 정확도 측정
ratio_list = [0.1, 0.3, 0.5, 0.7, 0.9]
lr_ratio_accu_list, cv_ratio_accu_list, ratio_check = ratio_acc(ohe_ac_X, ohe_ac_y, ratio_list)

# 테스트셋 비율 증가에 따른 정확도 변화 그래프
plt.figure(figsize=(7, 5))
plt.style.use('seaborn-dark')
plt.xlabel('The ratio of Testset')
plt.ylabel('Accuracy')
plt.title('Change of Accuracy by Increase of Testset Ratio')
plt.plot(ratio_check, lr_ratio_accu_list, color='crimson', alpha=0.73, linewidth=3, label="LR")
plt.plot(ratio_check, cv_ratio_accu_list, color='navy', alpha=0.73, linewidth=3, label="LR-CV")
plt.legend(["LR", "LR-CV"], loc="best");

- 결과 분석
    1. 테스트셋 비율 10%(4500명 학습/500명 예측)부터 90%(500명 학습/4,500명 예측)까지 Logistic Regression CV버전의 예측정확도 100% 유지
    2. 반면 제한값 C를 수동으로 적용한 버전은 테스트셋 비율 50%(2,500명 학습/2,500명 예측)부터 떨어지기 시작
    3. 이를 통해 수동버전에 비해 CV버전이 보다 일반화된 학습을 할 수 있음이 드러남 (적은 학습으로 많은 예측)
    4. 나아가 테스트셋 비율 90%(500명 학습/4,500명) 예측에서 보인 100% 정확도는 오버핏에 대한 반증
- 결론
    1. 최종 모델 - area_code(non-ordinal encoding) 외 16개 지표와 Logistic Regression CV로 학습한 버전
    2. 테스트셋 비율은 위 실험 전 최초 설정한 30%를 다시 적용 (보다 많은 수의 고객 예측 대비)
    3. 오버핏 검증을 마쳤고, 100% 정확도를 보이는 머신러닝 모델을 얻었으므로 딥러닝 모델 실험은 생략

- 실험 종료 (area_code열 활용 모델 오버핏 검증)

> # - 모델 전달(혹은 배달): 원클릭 고객 이탈 예측 서비스

### 1. 최종 고객이탈율 예측 모델

In [ ]:
# 최종 모델 학습기
def final_model(X_train, y_train, X_test, y_test, f, r): # f = 트레이닝 데이터 분리 수 / r = 랜덤시드
    from sklearn.linear_model import LogisticRegressionCV
    LR_CV = LogisticRegressionCV(cv=f, random_state=r).fit(X_train, y_train)
    prediction = LR_CV.predict(X_test)
    accuracy = get_accuracy(prediction, y_test)
    return LR_CV, prediction, accuracy

# 최종 모델 얻기 / 해당 모델로 예측정확도 재확인
final_X_train, final_X_test, final_y_train, final_y_test, _ = data_split(ohe_ac_X, ohe_ac_y, 0.3, 13) # 테스트셋 비율=30%, 랜덤시드=13
final_model, final_pred_cv, final_accu_cv = final_model(final_X_train, final_y_train, final_X_test,  final_y_test, 5, 13) # cross validation용 데이터 분리 수=5, 랜덤시드 =13

print('최종 모델을 활용한 고객이탈율 예측정확도: ', final_accu_cv*100, '%')

### 2. 오렌지텔레콤 고객관리 담당자용 - 최초 데이터 입력부터 최종 이탈 예측까지 이어지는 원클릭 서비스

In [ ]:
from __future__ import print_function # 출력 기능은 셀 첫 줄에 위치

# 최초 데이터 입력에서 최종 이탈 예측까지 이어지는 파이프라인
def data_to_predict(data, model=final_model, real=0):
    ## 0. 필요환경 구성 / 데이터 불러오기
    import os
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    import warnings
    warnings.filterwarnings('ignore', module='sklearn')
    from sklearn.preprocessing import StandardScaler
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.preprocessing import MaxAbsScaler
    from sklearn.preprocessing import LabelBinarizer
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.linear_model import LogisticRegressionCV
    data_path = ['../input/intelml101class1']
    data = pd.read_csv(filepath)

    ## 1. 데이터 전처리
    data_new = data.copy() # 비상용 원본 저장
    ac_col_unused = ['state', 'account_length', 'phone_number']
    ac_data = data_new.drop(columns=ac_col_unused)
    if real == 1:
        for column in ['intl_plan', 'voice_mail_plan']:
            ac_data[column] = lb.fit_transform(ac_data[column])
    else:
        for column in ['intl_plan', 'voice_mail_plan', 'churned']:
            ac_data[column] = lb.fit_transform(ac_data[column])
    ac_encoded = pd.get_dummies(ac_data.area_code, prefix=['area_code'], drop_first=True)
    all_ac_data = pd.concat([ac_data, ac_encoded], axis=1)
    all_ac_data.drop(['area_code'], axis=1, inplace=True)
    all_ac_data_mms = pd.DataFrame(MinMaxScaler().fit_transform(ohe_ac_data), columns = all_ac_data.columns)
    all_ac_data_mms.describe().T.tail(n=3); # 유닛테스트(비활성) - 스케일링 후 결과
    orange_X = all_ac_data_mms.copy()
    orange_y = orange_X.pop('churned')
    #print(orange_X.columns) # 유닛테스트(비활성) - X에 y 포함 여부
    
    ## 2. 고객 이탈 예측 - 최종 학습된 우리 모델 활용
    orange_prediction = final_model.predict(orange_X)
    orange_predict_proba = final_model.predict_proba(orange_X)
    
    ## 3. 이탈 예상 고객 목록
    data_final = data.copy()
    data_final.insert(loc=0, column='churn_probability', value=orange_predict_proba[:,1]*100)
    data_final = data_final.sort_values(by=['churn_probability'], ascending=False)
    churn_list = data_final[:int(sum(orange_prediction))]
    no_churn_list = data_final[int(sum(orange_prediction)):]
    
    return orange_prediction, churn_list, no_churn_list

[이렇게 써주세요]
1. 아래 회색 바탕에 영어로 쓰인 줄의 오른쪽 부분을 봐주세요. 예측을 원하는 고객 데이터의 이름을 (data_raw) 자리에 넣어주세요.
    - 이렇게요> data_to_predict(데이터 이름)
    - 데이터 이름은 영어만 쓰실 수 있어요 (영어회화 학원처럼요)
2. 시뮬레이션이 아닌 실제 예측을 원하시는 경우(과거 고객 이탈에 대한 정보가 없는 경우) 데이터 이름 뒤에 1을 붙여주세요.
    - 이렇게요> data_to_predict(데이터 이름, 1)
3. 마지막으로 시프트 + 엔터를 눌러주시면 끝이에요. 나머진 저희에게 맡겨주세요!

In [ ]:
# 여기서 시프트+엔터!
orange_prediction, churn_list, no_churn_list = data_to_predict(data_raw)

[이렇게 나왔어요]

In [ ]:
print('오렌지 담당자님!', len(data_raw), '분의 고객에 대한 이탈 예측을 원하셨죠?')
print('안타깝게도 이 중 이탈이 예상되는 분은', int(sum(orange_prediction)), '분이었어요. (저는 정말 0이 나오길 바랐어요)')
print('전체 중', sum(orange_prediction)/len(data_raw)*100.0, "%에 가까운 비율이에요. 과연 어떤 분들이 오렌지텔레콤과의 이별을 준비하시는지 아래에서 상세히 살펴볼게요.")

- 이탈 예상 고객 목록
    - 첫째로, 이별(이탈보다 어감이 나은 것 같아요) 가능성이 높은 분들이에요.
    - 우선 가장 위험한 세 분만 모셔봤어요. 왼쪽부터 이별 확률이 높은 순서예요.
        - 표 2번째 줄의 'churn_probability'가 이탈 가능성이예요. 100이 가장 높고 0이 가장 낮아요. 단위는 %이고요.
    - 시뮬레이션을 하셨다면 마지막에 churned(실제 이탈) 항목이 있을 거고요. 실제 예측을 원하셨다면 없을 거예요.

In [ ]:
churn_list.head(n=3).T

- 비이탈 예상 고객 목록
    - 이번엔 이별 가능성이 낮은 분들이에요.
    - 위와 마찬가지로 왼쪽부터 이별 확률이 높은 순서예요. 가장 오른쪽에 있는 분이 오렌지텔레콤을 가장 사랑하는 분이네요.

In [ ]:
no_churn_list.tail(n=3).T

- 이렇게 어떤 분들이 이별 가능성이 높고 어떤 분들은 그렇지 않은지 알아봤어요.
    - 아는 것이 힘이지만 알기만 하고 끝나면 변화가 없겠죠? 그래서 준비했어요. [프로젝트 진달래꽃]

### 3. 프로젝트 진달래꽃 (고객 이탈 방지 대안) - 진행 중
    1. 총이용금액, 계약 유지기간을 고려한 고객별 가치 측정
        - 총이용금액 : 계약 유지기간 = 0.7 : 0.3 (약 2.33 : 1) 가중치 적용
            - 산출 근거 - 비이탈 대비 이탈 고객의 높은 총이용금액 / 비슷한 계약 유지기간 (아래 히스토그램 대조 참고)
    2. 이탈율과 고객가치를 고려한 대응시급성 지표 산출
        - 이탈율 : 고객가치 = 0.85 : 0.15 (약 5.67 : 1) 가중치 적용 
            - 기존고객 유지비용 대비 신규고객 유입비용의 크기 (참고문헌 링크 필요)
    3. 대응시급성에 따른 이탈 예상 고객 순서 재배치
        - 결과: 707명 중 703명의 순서 변경 (약 99.43%)

- 고객가치 산출 시 '총이용금액 : 계약 유지기간 = 2.33 : 1' 가중치 적용 근거

총이용금액 기준 비이탈 / 이탈 예상고객 분포 대조

In [ ]:
# 1. 이탈/비이탈 예상 고객 총이용금액 확인
charge_list = []
for i in churn_list.columns:
    if 'charge' in i:
        charge_list.append(i)
total_charge_no_churn = no_churn_list[charge_list].sum(axis=1)
total_charge_churn = churn_list[charge_list].sum(axis=1)

# 2. 히스토그램
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12.5, 5))

plt.subplot(1, 2, 1)
plt.xlabel('Total Charge ($)')
plt.ylabel('Frequency')
plt.title('Frequency by Total Charge')
plt.hist(total_charge_no_churn, color='darkcyan', alpha=0.31)
plt.hist(total_charge_churn, color='darkcyan')
plt.title('No Churn vs. Churn')

plt.subplot(1, 2, 2)
plt.xlabel('Total Charge ($)')
plt.ylabel('Frequency')
plt.title('Frequency by Total Charge')
plt.hist(total_charge_churn, color='darkcyan')
plt.title('Churn Only')

fig.tight_layout()

- 비이탈 예상 고객 대비 이탈 예상 고객의 분포가 우측으로 치우쳐있음 (더 많은 요금을 지불)

계약 유지기간 기준 비이탈 / 이탈 예상고객 분포 대조

In [ ]:
# 히스토그램
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12.5, 5))

plt.subplot(1, 2, 1)
plt.xlabel('Account Length (month)')
plt.ylabel('Frequency')
plt.title('Frequency by Account Length')
plt.hist(no_churn_list.account_length, color='darkcyan', alpha=0.31)
plt.hist(churn_list.account_length, color='darkcyan')
plt.title('No Churn vs. Churn')

plt.subplot(1, 2, 2)
plt.xlabel('Account Length (month)')
plt.ylabel('Frequency')
plt.title('Frequency by Account Length')
plt.hist(churn_list.account_length, color='darkcyan')
plt.title('Churn Only')

fig.tight_layout()

- 계약 유지기간 - 비이탈 예상 고객 대비 이탈 예상 고객의 분포가 유사함 (계약 유지기간에 있어 이탈 예상 고객 특이점 없음)

#### 결론
- 이탈 예상 고객을 잃음으로써 오렌지텔레콤이 입게 될 손해는 (비이탈 예상 고객 대비) 계약 유지기간의 차이가 아닌 총이용금액의 차이로 인해 더 커짐
- 즉 이탈 예상 고객의 가치를 결정하는 데 있어 두 지표의 중요도가 달라야함 (총이용금액 > 계약 유지기간)
- 현재 적용 가중치 비율 2.33 : 1 (총이용금액 : 계약 유지기간)

- 대응시급성 산출 시 '이탈 가능성 : 고객가치 = 5.67 : 1' 가중치 적용 근거 - 작성 중

In [ ]:
# 총이용금액, 계약 유지기간, 이탈 가능성을 고려한 대응시급성 산출기
def probability_to_emergency(churn_list=churn_list, charge_weight=0.7, churn_weight=0.85):
    churn_list_new = churn_list.copy() # 반복실험용 원본데이터 저장
    # 0. 총이용금액 열 추가
    charge_list = []
    for i in churn_list_new.columns:
        if 'charge' in i:
            charge_list.append(i)
    churn_list_new.insert(loc=1, column='total_charge', value=churn_list_new[charge_list].sum(axis=1))

    # 1. 고객가치 계산에 필요한 데이터 구성
    churn_list_new.churn_probability = churn_list_new.churn_probability/100.0
    emer_list = pd.DataFrame(MinMaxScaler().fit_transform(churn_list_new.iloc[:,1:4].drop('state', axis=1)),
                             columns=['total_charge', 'account_length'], index=churn_list_new.index)
    emer_list = emer_list.join(churn_list_new.churn_probability)

    # 2. 총이용금액 / 계약 유지기간 열에 가중치 적용하여 고객가치 계산 후 이탈 예상 리스트에 추가
    emer_list['customer_value'] = emer_list.total_charge*(charge_weight) + emer_list.account_length*(1-charge_weight)
    churn_list_new.insert(loc=0, column='customer_value', value=emer_list.customer_value)

    # 3. 이탈 가능성 / 고객가치 열에 가중치 적용하여 대응시급성 계산 후 이탈 예상 리스트에 추가
    emer_list['emergency'] = emer_list.churn_probability*churn_weight + emer_list.customer_value*(1-churn_weight)
    churn_list_new.insert(loc=0, column='emergency', value=emer_list.emergency)

    # 4. 대응시급성 지표에 따라 전체 열 재배치 / 소수점 수치 백분율로 바꾸기
    churn_list_new = churn_list_new.sort_values(by=['emergency'], ascending=False)
    churn_list_new.iloc[:,:3] = churn_list_new.iloc[:,:3]*100
    churn_list_new = churn_list_new.round(2)
    
    return churn_list_new

### 3-1. 대응시급성 분석기

[이렇게 써주세요] - 오렌지 담당자님! 이번에도 원클릭이에요.
1. 위에서 하셨던 것처럼 아래 화면에서 시프트+엔터를 눌러주세요
2. 그러면 각 고객분의 가치가 얼마나 높은지
3. 그리고 이 가치에 따라 어떤 분들이 가장 시급한 대처를 필요로 하는지 보여드릴게요

In [ ]:
new_churn_list = probability_to_emergency(churn_list=churn_list, charge_weight=0.5, churn_weight=0.85)

[이렇게 나왔어요]

In [ ]:
new_churn_list.drop('state', axis=1).head(5).T.head(5)

- 오렌지 담당자님의 빠르고 정확한 이별방지대응이 필요한 고객분들이예요. (이별은 막을 수 있어요!)
    - 첫째로, 총이용금액과 계약 유지기간을 고려해서 고객 한 분 한 분의 가치를 파악했어요.
    - 둘째로, 이렇게 얻은 고객가치와 앞서 저와 함께 알아보신 이별가능성을 고려해서 대응의 시급성을 계산했어요.
    - 셋째로, 이 시급성을 기준으로 이별이 예상되는 707명의 고객분들의 우선순위를 다시 매겨봤어요.
- 이 중 대응이 가장 시급한 다섯 분을 모셨어요. 왼쪽부터 이별을 막기 위한 노력이 먼저 필요한 순서예요.
- 표 2번째 줄의 'emergency'가 바로 그 대응시급성이예요. 100이 가장 높고 0이 가장 낮아요. 단위는 %이고요.
- 이번에는 표 안의 수많은 항목들 중 시급성 파악에 가장 필요한 다섯 항목만 꼽아봤어요.

[이렇게 변했어요]

In [ ]:
# 이탈 예상 고객 순위 변동 비율
num_change = sum(churn_list.index != new_churn_list.index)
ratio_change = num_change/len(churn_list)*100
ratio_change = np.around(ratio_change, 2)

print("이탈가능성(churn_probability)에서 대응시급성(emergency)을 기준으로 많은 분들의 순위가 바뀌었어요.")
print("정확하게는", num_change, "분이고요. 총", len(churn_list), "분의 이별 예상 고객 중", ratio_change, "%에 가까워요.")
print("즉 이런 과정을 거치지 않은 채 이별 가능성만을 기준으로 고객분들께 접근했다면,")
print("많은 시간과 노력을 잘못된 순서에 따라 썼을 수 있다는 의미랍니다.")

[진달래꽃]<br>

나 보기가 역겨워 가실 때에는<br>
말 없이 고이 보내 드리오리다<br>
영변에 약산 진달래꽃<br>
아름 따다 가실 길에 뿌리오리다<br>
가시는 걸음 놓인 그 꽃을<br>
사뿐히 즈려밟고 가시옵소서<br>
나 보기가 역겨워 가실 때에는<br>
죽어도 아니 눈물 흘리오리다

잠깐 우리 프로젝트의 의미를 떠올려보았어요.<br>
하지만! 이렇게 고이 보내드릴 순 없어요.<br>
고객 한 분 한 분 모두 소중하니까요.<br>
그래도 막무가내로 매달릴 순 없겠죠?<br>
우아하게 하지만 정확하게 붙잡습니다.<br>
사랑에도 전략이 필요하니까요.

[이래도 가실테요] - 이탈 방지 전략 (작성 중)
    1. 행동 - 적용 가능한 대안 설정 (예: 요금할인, 부가서비스/쿠폰/상품권 제공, 안부문자)
    2. 자원 - 고객별 대안에 투입할 수 있는 총비용의 최대치 파악 (고객가치 기준)
    3. 전략 - 대안별 제시 시기 및 순서 전략화
    4. 예측 - 전략 효과 예측 (시뮬레이션)
    5. 선정 - 최대 보상(고객 비이탈)을 산출하는 최적화 전략 선정 후 오렌지텔레콤에 전달

- 본 자료는 [Intel](https://www.intel.com/content/www/us/en/homepage.html)의 [AI Developer Program](https://software.intel.com/en-us/ai) - [AI Academy - Machine Learning](https://software.intel.com/en-us/ai/courses/machine-learning) 2주차 내용 중 일부에 기반하여 작성되었습니다.


In [ ]:
# 잠시 쉬는 줄 (여긴 읽지 마시고 눈을 쉬게 해주세요!)

>  # 2. 배민라이더스 앱 리뉴얼 효과 분석 제안서
    [흐름]
    - 정의 및 구체화
        - 상황 설정
        - 용어 정의
        - 목표 재정의
        - 문제 정의
        - 문제 구체화
        - 필요 자원/자료 정의
    - 조사 및 분석
        - 조사/분석 실행 전 사내 조사
        - 조사/분석 실행 시 유의사항
        - 단계별 조사/분석 실행
        - 각 후보안에 대한 판단 근거
    - 해결 및 평가
        - 최종 해결안 선정
        - 최종 해결안 실행
        - 결과 확인 및 평가

> # - 정의 및 구체화

## [상황 설정]

- 현재 우아한청년들은 라이더들이 주문을 받고 배달을 하는 데 활용하는 '배민라이더스' 앱의 리뉴얼을 계획 중입니다.
- 목표는 배달서비스 품질 개선이며 이를 위한 여러분의 제안을 기다립니다.

## [용어 정의]

1. 라이더
	- 배민라이더스에 소속되어 배달 임무를 수행하는 자 
	- 그 중에서도 배달 임무에 배민라이더스 앱을 적극 확용하는 자
	- 적극 라이더: 모든 배달 주문을 배민라이더스 앱을 통해 받고 처리하는 자
	- 비적극 라이더: 비공식적인 경로를 통한 주문을 받아 수익을 창출하는 자
	- 비공식 경로: 배민라이더스 앱 이외 모든 경로 (다른 앱, 문자, 전화 등)
2. 배민라이더스 앱
	- 우아한형제들 개발팀을 통해 개발/배포된 배민라이더스 라이더용 배달 지원 공식 앱
	- 비공식 앱: 위 방법이 아닌 비공식 경로를 통해 배포된 유사 앱
	- 비공식 경로: 개발자 개인, 애플 탈옥 마켓, 안드로이드 블랙마트 등을 통한 배포
3. 리뉴얼
    - 현재 버전의 앱에 속하는 기능/외형의 일부를 변경하는 활동
	- 현재 버전의 앱에 속하지 않는 기능/외형을 추가하는 활동
        - 현재 버전에 속하지 않는 기능/외형이라도 과거 배포된 버전과 앱 전체적으로 동일한 조합이라면 리뉴얼에 속하지 않음
	- 앱 전체가 아닌 부분은 과거의 기능/외형을 참고 및 활용할 수 있음
4. 배달서비스 품질
	- 정량적: 
            - 주문 수
                - 라이더별 주문 수, 배민라이더스 전체 주문 수
                - 우아한청년들 서비스(B마트 포함) 전체 주문 수
                - 우아한형제들 서비스(배달의민족 포함) 전체 주문 수
            - 위험 요소
                - 배달 소요 시간
                - 배달 중 사고 발생 비율 
		    - 만족도 수치
                - 앱스토어/플레이스토어 고객 리뷰(5점 만점)
                - 고객 리뷰 수, 이 중 긍정/부정 리뷰의 비율
		    *위 모든 지표의 측정 및 평가는 30일을 기준으로 함
	- 정성적: 
            배민라이더스 고객 만족도 설문조사, 미스터리오더(미스터리쇼퍼의 배달 버전)를 통한 항목별 평가 취합
5. 개선
	- 주문 수 / 리뷰 수 / 긍정 리뷰 비율: 기존 수치 대비 10% 이상의 증가
	- 배달 시간 / 사고 비율: 기존 수치 대비 10% 이상의 감소
	- 10% 미만의 변화: 계절성, 이벤트성, 일반적인 증감 주기 등에 따른 변화로 간주
	- 정성적 지표: 정량적 지표 개선에 대한 검증자료로 활용
        - 전체 정성적 의견의 50% 이상이 부정적일 경우, 정량적 개선이 있더라도 서비스 품질 전체는 개선되지 않았다고 판단

## [목표 재정의]

- 우아한청년들은 배달서비스 품질 평가 항목 중 1개 이상의 개선을 위해
- 배민라이더스 공식 앱의 기능적/외형적 변화를 계획하고 있습니다. 
- 앱의 사용자는 배민라이더스 소속 공식 라이더들이며
- 배달 주문을 받고 배달 경로를 확인하는 등 전체 배달 프로세스에 활용합니다.

## [문제 정의]

1. 리뉴얼이 필요한가?
2. 필요하다면, 최우선 목적은 없는 기능/외형의 추가인가? 있는 기능/외형의 변경인가?
3. 필요하지 않다면, 리뉴얼을 하지 않고 배달서비스 품질 개선을 이룰 수 있는 방법은 무엇인가?

## [문제 구체화]

1. 리뉴얼의 필요성
	- 현재 앱은 얼마나 만족스러운가?
	- 과거 버전의 앱과 비교하면 얼마나 더/덜 만족스러운가?
	- 경쟁 배달업체의 앱과 비교하면 얼마나 더/덜 만족스러운가?
	- 일반 지도 앱과 비교하면 얼마나 더/덜 만족스러운가?
	- 라이더들이 평소 사용하는 다른 앱(유튜브 등)과 비교하면 얼마나 더/덜 만족스러운가?
2. 리뉴얼의 목적
	- 필요하지만 없는 기능은 무엇인가?
	- 있지만 쓰이지 않는 기능은 무엇인가?
	- 쓰이지만 제대로 작동하지 않는 기능은 무엇인가?
	- 앱 다운을 일으키는 기능은 무엇인가?
	- 외형적인 부분에서 우아한청년들의 다른 서비스(B마트), 우아한형제들 전체 브랜드(특히 배달의민족)와 얼마나 조화로운가?
3. 리뉴얼 없이 개선
	- 라이더에게 앱 사용에 대한 안내가 덜 된 것은 아닌가?
	- 라이더가 앱을 활용하는 데 따른 보상/의무가 없거나 부족하진 않은가?
	- 앱을 통해 얻은 데이터를 앱 자체가 아닌 다른 서비스에 활용할 수는 없는가?
	- 다른 서비스를 통해 얻은 데이터를 배민라이더스 앱에 활용할 수는 없는가?
        - 위의 방법이 있고 그로 인한 서비스 개선이 기대된다면, 그 적용을 위해 리뉴얼이 필요한가?
        - 데이터 연결 혹은 직접 활용을 통해 구현할 수는 없는가?

## [필요 자원/자료 정의]

1. 리뉴얼의 필요성
	- 현재/과거 앱 만족도 조사
        - 앱스토어/플레이스토어 리뷰 선호도 분석
        - 라이더 설문조사
	- 경쟁 배달업체 앱 만족도 조사
        - 앱스토어/플레이스토어 리뷰 선호도 분석
        - 경쟁업체 라이더용 앱 사용 후 결과 정리 (라이더 등록없이 앱만 사용 가능한지 파악)
            - 등록이 필요하다면 배민라이더스 소속 라이더 중 해당 앱 사용 가능한 라이더 파악
                - 없다면 외부 라이더 중 해당업체에 소속되지 않았으나 앱 사용 가능한 라이더 탐색 후 설문 의뢰
	- 일반 지도 앱 만족도 조사
        - 앱스토어/플레이스토어 리뷰 선호도 분석
        - 배민라이더스 소속 라이더 중 일부(30명 내외)를 선별하여 배달 시 일반 지도 사용 후 결과 보고 의뢰
	- 일반 앱 만족도 조사
        - 앱 애니(App Annie) 등의 앱 분석 서비스를 통한 앱 사용성/선호도 트렌드 파악
        - 라이더 설문조사 (가장 많이 사용하는 앱, 그 앱에 대한 선호도)
2. 리뉴얼의 목적
	- 국내 경쟁업체 라이더용 앱 1:1 대조 기능/외형 분석 (빅3)
	- 국외 배달업체 라이더용 앱 1:1 대조 기능/외형 분석 (중국, 미국, 아시아, 유럽 각 지역 빅1)
	- 라이더들의 앱 사용패턴 분석
		- 배달 건수 대비 접속 횟수, 최다 사용 기능, 1회 접속 시 머무는 시간 등
    - 배민라이더스 앱 자체 오류 분석
        - 전체 접속 횟수 중 다운 비율, 바로 직전 실행된 기능, 보여진 화면 파악
	- 배달의민족 앱 사용자 중 일부를 선정, 배민라이더스 앱 사용 후 배달의민족 앱과의 차이에 관한 설문 의뢰
        - 배달 경험이 있는 사용자를 우선 선발 (설문 완료 시 포인트 지급)
	- 배민라이더스 라이더 중 일부를 선정, 배달의민족 앱 사용 후 배민라이더스 앱과의 차이에 관한 설문 의뢰
        - 이미 배달의민족 앱 사용 경험이 있더라도 각 설문 항목에 집중하여 다시 사용하도록 유도
3. 리뉴얼 없이 개선
    - 라이더 앱 사용률 개선
        - 전체 라이더 대비 배민라이더스 앱 사용 라이더 비율 측정
        - 앱 사용 라이더 중 적극 사용 라이더 비율 측정
        - 비적극 라이더 대상 배민라이더스 앱 사용 패턴 분석
            - 적극/일반 라이더 사용 패턴과의 차이 파악
        - 비적극/비사용 라이더 대상 배달 시 이용하는 다른 앱 조사 및 그 패턴 분석 (설문)
    - 데이터 연결을 통한 서비스 품질 개선
        - 라이더들의 최적 배달 동선을 '가장 빨리 맛집 찾길'이란 이름의 오픈데이터로 일반 고객에게 공개하는 방안 검토 
            - 점심, 저녁 등 사람이 몰리는 때에 보다 일찍 도착하여 줄 서서 기다리는 시간 감소
        - B마트 주문이력 중 음식 관련 데이터를 통해 한식러/양식러/중식러/일식러/잡식러로 고객을 분류
            - 이 '각식러 분포도'를 주문자 가까이 위치한 각 종류의 맛집 사장님과 라이더에게 공유하는 방안 검토
                - 사장님 재고관리 및 라이더 대기지역 최적화 시 활용
        - '가장 빨리 맛집 찾길' 및 '각식러 분포도' 적용 전 비용 대비 효과를 예측하여 앱 리뉴얼보다 효율이 클 경우
            - 앱 리뉴얼 전 적용 > 결과 측정 > 서비스 품질 개선 여부 확인 
                - 앱 리뉴얼 진행 여부 재판단 (기준 성과 및 앱 환경 일부가 바뀌었으므로)

> # - 조사 및 분석

## [조사/분석 실행 전 사내 조사]

1. 사내 설문조사를 통해 '리뉴얼 진행, 리뉴얼 없이 대안 진행, 대안 진행 후 리뉴얼, 리뉴얼 후 대안 진행' 각 안에 대한 사업부별 직군별 직책별 의견 수렴
    - 일반적인 앱 리뉴얼과 이외의 대안에 대한 임직원들의 지지도 파악
2.  배민라이더스 외 사업부의 기존 데이터를 통해 해결가능한 과제가 있는지 파악 후 해당 부서에 협조 가능성 확인
3.  특히 대안 중 '가장 빨리 맛집 찾길' 및 '각식러 분포도'를 진행하는 경우 우아한형제들 전사 브랜드팀 및 B마트팀과 사전 협의 필수

## [조사/분석 실행 시 유의사항]

1. 리뉴얼의 필요성
	- 라이더 만족도 조사 시 앱 리뉴얼 계획을 알리지 않고 비정기 설문조사 형태로 의뢰
        - 새 것에 대한 막연한 반발/기대 차단
    - 경쟁업체 앱 만족도 분석 시 각 설문자의 소속(우리 라이더, 외부 라이더)에 따라 다른 가중치를 적용
        - 이해관계에 따른 선입견 최소화
	- 일반 앱 만족도 조사 시 라이더의 개인정보와 밀접히 관련된 앱(예: 페이스북)은 제외
2. 리뉴얼의 목적
	- 국외 배달업체 라이더용 앱 분석 대행자 선정 시 라이더 경험 확인
        - 사전 체크리스트 제공하여 국내 분석 내용과 지표 및 입력값의 범위 일치화
	- 배달의민족 앱 사용자의 배민라이더스 사용 의뢰 전 해당 고객의 앱 내 접근범위 설정 필요
	- 배민라이더스 라이더가 배달의민족 앱 사용 시 이해관계자(예: 친한 사장님)를 통하여 주문하지 않도록 경고
3. 리뉴얼 없이 개선
	- 앱 비적극 사용 라이더의 다른 앱에 대한 솔직한 답변을 얻기 위해
        - 우리 앱 비사용에 따른 불이익이 없고, 있다 해도 소급하여 적용되지 않음을 안내
	- '가장 빨리 맛집 찾길'로 라이더 배달동선 공개 결정 시 라이더 개인 추적이 불가하도록 배달시각 정보 삭제
	- '각식러 분포도' 공유 결정 시 개별 고객의 식별이 불가하도록 변환하되
        - 한식/양식/일식/중식/잡식 선호도는 쉽고 명확하게 시각화하여 바쁜 조리/배달 중에도 한 눈에 파악할 수 있도록 제공

## [단계별 조사/분석 실행]
1. 리뉴얼의 필요성에 대한 조사/분석 우선 실행
2. 리뉴얼의 필요성이 인정된다면, 리뉴얼의 목적에 대한 조사/분석 진행
    - 리뉴얼의 목적에 대한 조사/분석 후 추가 조사/분석을 위한 예산과 시간이 있다면
        - 리뉴얼 이외 대안에 대한 조사/분석 진행
3. 리뉴얼의 필요성이 인정되지 않는다면, 리뉴얼 이외 대안에 대한 조사/분석 진행

## [각 후보안에 대한 판단 근거]

1. 리뉴얼
	- 진행 근거
        - 1개 이상의 필수기능이 부재하거나 50% 이상의 확률로 오작동
            - 필수기능: 주문 접수, 배달 동선 파악, 결제 등 (없으면 배달 업무가 불가능한 기능)
        - 과거버전/경쟁앱/일반앱 (중 2개 항목 이상) 대비 80% 이하의 사용자 만족도        
	- 비진행 근거
        - 2개 이하의 부가기능이 부재하거나 50% 이상의 확률로 오작동
            - 부가기능: 앱 내 전화/문자 등 (없어도 다른 앱 활용을 통해 배달 업무가 가능한 기능)
        - 과거버전/경쟁앱/일반앱 (중 2개 항목 이상) 대비 120% 이상의 사용자 만족도        
2. 리뉴얼 진행 시, 최우선 과제
	- 기능성 우선 개선 근거
        - 2개 이상의 필수기능이 부재하거나 50% 이상의 확률로 오작동
	- 외형성 우선 개선 근거
        - 배달의민족 사용자의 배민라이더스 앱 사용 후기와 배민라이더스 라이더의 배달의민족 앱 사용 후기 모두에서
            - 배달의민족 앱 대비 배민라이더스 앱의 외형에 대한 부정적 응답이 50% 이상
                > 예: 배민라이더스 앱이 훨씬 덜 이뻐요!, 배민 앱 안 같아요!
3. 리뉴얼 이외 대안 진행 시, 적용 과제
    - 라이더 앱 사용 독려 방안 근거
        - 보상: 앱 사용 횟수와 전체 배달 수 일치 비율에 따른 성과제 도입/보너스 지급이 가능한 경우
        - 의무: 배민라이더스 신규 라이더 등록 시 앱 사용에 관한 의무(100% 활용)를 계약서에 명시하는 것이 합법적인 경우
        - 처벌: 비공식 앱 사용/비공식 배달 행위 3회 이상 발견 시 라이더 자격을 박탈하는 것이 근로계약 상 문제가 되지 않는 경우
	- 가장 빨리 맛집 찾길: 배민라이더스 맛집 배달동선 오픈데이터화 근거
        - 사내 규정 및 개인정보보호법 상 해당 데이터의 공개가 가능한 경우
        - 라이더 개인 식별이 불가한 경우
        - 맛집 길 찾기에 대한 수요가 유의미하게 확인된 지역이 서울 구 기준 30% 이상인 경우 (본 지역들 우선 적용)
            - 유의미한 수요: 맛집 당 월 90회(일 평균 3회) 이상의 길 찾기 검색 수
	- 우리 동네 각식러 분포도: B마트 식료품 주문 기록을 지도화하여 맛집 사장님과 배민라이더스 라이더에게 공유 근거
        - 우아한청년들 사업부 간 데이터 공유가 가능하고 이 중 일부를 사장님과 라이더에게 공유할 수 있는 경우
        - 유형별 뚜렷한 분류가 가능할만큼의 데이터가 쌓인 경우
        - 고객 개인 식별이 불가한 경우
        - 공유 결정 시, 배민라이더스를 최근 30일 간 3회 이상 이용한 B마트 고객(파워식러)의 구매 이력에 더 큰 가중치를 두어 분포도 제작

> # - 해결 및 평가

## [최종 해결안 선정]

1. 각 후보안에 대한 조사/분석을 통해 가장 많은 근거가 뒷받침된 해결안을 작성
    - 해결안: 과제별, 항목별 실행안을 엮은 하나의 해결 시나리오
        - 예: 앱 리뉴얼 진행 > 우아한형제들 전체 브랜드와의 외형 통일화 최우선 > 부가기능 사용성 개선 > 각식러 분포도 진행
2. 작성된 해결안과 선정 근거를 문서화하여 보고 (선정 뒤 3일 이내)
3. 의사결정자 및 관계자에게 프레젠테이션 형태로 구두 제안
4. 의사결정자 전원과 관계자 50% 이상의 동의 하에 최종 해결안 선정
    - 동의 조건 미충족 시 추가 조사/분석 후 다시 제안

## [최종 해결안 실행]

1. 관련부서에 실행 요청 (데이터과학 업무의 경우 직접 실행)
2. 실행의 진행도/이행의 정확도/예상 외 변수 모니터링
3. 모니터링 중 필요하다고 판단되거나 실행자가 요청하는 경우 추가 조사/분석
4. 추가 분석 결과 갱신해야 할 사항이 있다면 실행자에게 전달 후 적용 여부 점검
5. 최종 실행 완료 확인

## [결과 확인 및 평가]

1. 최우선 성과 확인 및 분석
	- 최종 목표를 달성하였는가?
        - 비용/시간은 얼마가 들었는가? 예측 범위 내인가?
            - 프로젝트 전체 비용/시간 중 각 과제에 소요된 비용/시간은 얼마씩인가?
	- 최우선 과제를 해결하였는가?
        - 비용/시간은 얼마가 들었는가? 예측 범위 내인가?
            - 과제 전체 비용/시간 중 각 항목에 소요된 비용/시간은 얼마씩인가?
	- 최우선 과제를 해결하였지만 최종 목표는 달성하지 못했다면 그 이유는 무엇인가?
		- 목표에 맞는 과제 선정에 대한 의문
	- 이후 다른 업무를 위해 위의 문제를 어떻게 해결할 수 있는가?
		- 과제 실행 전 목표와의 적합성 검증을 위한 절차 확립
            - 예: 유사 과제로 유사 목표를 달성한 과거 사례 조사 (사내/사외/해외 데이터 분석)
2. 추가적 성과 확인 및 분석
	- 기대하였고 실제로 달성된 성과는 무엇인가?
	- 기대하지 않았지만 달성된 성과는 무엇인가?
	- 추가 성과가 없다면 그 이유는 무엇인가? 
		- 제시된 해결안의 다른 문제로의 응용가능성에 대한 의문
	- 추가 성과를 달성하기 위해 평소 어떤 활동이 필요한가?
		- 사업부, 직군, 직책 간 연계된 데이터 및 각각 진행 중인 프로젝트에 대한 의견 공유를 활성화할 수 있는 방안 마련
            - 예1: '뭐 이런 데이털 다' 공모전
                 1. (아직은) 아무도 관심없지만 우리한테는 무지하게 흥미로운/중요한 부서별 데이터 대공개 (공개 가능한 부분만)
                 2. 관심있어요! 재미있어요! 더 알려주세요! 투표 진행
                 3. 가장 핫한 반응을 이끌어낸 세 데이터에 대한 보다 세부적인 데이터 공개 (참가자 동의 시)
                 4. 투자할래요!(사업성을 봄) 나를 가져요!(같이 이 데이터를 다루고 싶음) 그만 말해요!(진짜 될 것 같아서 나만 알고 싶음) 투표 진행
                 5. 최다 득표한 데이터 대상 사업성 검증 및 사내 벤처 발족 시 투입 희망 인원 파악
                 6. 정말 되겠다 싶을만큼의 검증과 인원이 모인 경우, 데이터 심층분석 및 사업화 진행
            - 예2: '프로젝트 런(learn)웨이' 게시판
                 1. 각 부서에서 진행 중인 프로젝트에 코드네임과 캐릭터를 설정 (맛집찾길이, 앱리뉴얼라, 정산탈태 등)
                 2. 각 프로젝트 진행 정도에 따라 이들이 달리기를 하는 모습의 게시판을 사내 복도에 설치
                 3. 과정 중 이 캐릭터들이 무엇을 배우고 있는지 각 구성원들이 배움 스티커 부착
                 4. 다른 부서원들이 댓글 말풍선 부착
                 5. 프로젝트 진행 중 어려움을 겪는 부서는 서서 달리지 않고 넘어져 있는 모습의 캐릭터로 도움 요청
3. (위 모든 과정을 거친 뒤) 최종 목표, 최우선 과제 해결, 기대 성과, 추가 성과를 모두 달성한 경우
	- (최)우수 조사/분석/실행 사례로서 문서화하여 사내 공유
    - 관련자들의 인터뷰를 모아 '우리는 어쩌다 이렇게 잘하고 말았나' 영상 제작 및 공유
    - 사내 반응이 (지나치게) 뜨거울 경우, 외부 공개용 특별강좌로 확장 (사업기밀, 개인정보 제거 후)
    - 외부 반응 역시 (말도 안 되게) 뜨거울 경우
        - (못 이기는 척) 문서화된 내용, 관련자 인터뷰, 유명인사(우형인 가능) 추천문을 엮어 출간
        - 책 수익금으로 팀 회식
            - 우리 팀만으로 감당이 안 될만큼 수익금이 많다면, 우형 전체 회식!

제안 완료

In [ ]:
# 두 문서(인텔을 이기다, 배민라이더스 앱 리뉴얼 효과 분석 제안서)와 관련된 질문이 있다면 댓글로 알려주세요
## 그동안 kaggle 속에 꼭꼭 숨어 지냈는데 이번 기회에 처음으로 이렇게 공개활동을 하네요
### 워낙 고수분들이 많은 프로의 세계라 겁나지만 설레기도 합니다

긴 글 읽느라 고생 많으셨습니다. 기운찬 연초 보내세요 : ^ )